## Install Additional Packages

In [ ]:
!pip install transformers==4.49
!pip install sentence-transformers
!pip install langchain
!pip install langchain_community
!pip install faiss-cpu
!pip install ragatouille
!pip install "unstructured[all-docs]"
!pip install rouge-score
!pip install bert-score

  Using cached unstructured-0.18.3-py3-none-any.whl.metadata (24 kB)
  Using cached python_magic-0.4.27-py2.py3-none-any.whl.metadata (5.8 kB)
  Using cached emoji-2.14.1-py3-none-any.whl.metadata (5.7 kB)
  Using cached python_iso639-2025.2.18-py3-none-any.whl.metadata (14 kB)
  Using cached langdetect-1.0.9.tar.gz (981 kB)
  Preparing metadata (setup.py) ... done
  Using cached rapidfuzz-3.13.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (12 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 120.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.5/112.5 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 5.2 MB/s eta 0:00:00


## Imports

In [ ]:
import os
import numpy as np
import nltk
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn.functional as F
import gc
import re
import json
import time
import inspect

from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.llms import HuggingFacePipeline
from langchain_community.document_loaders import (
    DirectoryLoader,
    UnstructuredMarkdownLoader,
)

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document as LangchainDocument
from langchain_community.vectorstores import FAISS
from langchain.embeddings.base import Embeddings
from langchain.prompts import PromptTemplate
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from tqdm import tqdm
from typing import Optional, List, Tuple, Any, Union, Pattern, Dict, Callable
from huggingface_hub import login
from ragatouille import RAGPretrainedModel
from datetime import datetime
from functools import wraps
from rouge_score import rouge_scorer
from bert_score import score
from requests.exceptions import HTTPError

nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')
pd.set_option("display.max_colwidth", None)

/tmp/ipython-input-2-1030601560.py:32: UserWarning: 
********************************************************************************
RAGatouille WARNING: Future Release Notice
--------------------------------------------
RAGatouille version 0.0.10 will be migrating to a PyLate backend 
instead of the current Stanford ColBERT backend.
PyLate is a fully mature, feature-equivalent backend, that greatly facilitates compatibility.
However, please pin version <0.0.10 if you require the Stanford ColBERT backend.
********************************************************************************
  from ragatouille import RAGPretrainedModel
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.


## Login to HuggingFace

In [ ]:
login()

## Tokenisers

In [ ]:
# NOTE: only one tokenizer was implemented in these experiments due to time constraints. Future work would implement multiple tokenisers to compare performance between them

# LARGE_TOKENISER_MODEL_NAME = "????"
MEDIUM_TOKENISER_MODEL_NAME = "Alibaba-NLP/gte-Qwen2-1.5B-instruct"
# SMALL_TOKENSIER_MODEL_NAME = "intfloat/multilingual-e5-large-instruct"

## LLMs

In [ ]:
# Defining model names in variables as re-used throughout file

LARGE_FINETUNED_LLM_MODEL_NAME = "johndennehy101/Mistral-7B-Instruct-v0.3-finetune-irish-citizen-info-v1"
LARGE_LLM_MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.3"
MEDIUM_LLM_MODEL_NAME = "meta-llama/Llama-3.2-3B-Instruct"
SMALL_LLM_MODEL_NAME = "Qwen/Qwen2.5-1.5B-Instruct"

## Prompt Types

In [ ]:
# Defining prompt types

ZERO_SHOT_PROMPT_TYPE = "zero shot"
PERSONA_PROMPT_TYPE = "persona"
FEW_SHOT_PROMPT_TYPE = "few shot"
INSTRUCTION_PROMPT_TYPE = "instruction"
CHAIN_OF_THOUGHT_PROMPT_TYPE = "chain of thought"
TREE_OF_THOUGHT_PROMPT_TYPE = "tree of thought"

## Common variables

In [ ]:
# These were implemented to ease testing (for chunk size for RAG & path to selected vector db)

CHUNK_SIZE = 512
VECTOR_DB_PATH = "/content/faiss_vector_store_Alibaba-NLP_gte-Qwen2-1_512_chunk"

## Utility Functions

In [ ]:
def get_device() -> str:
    """
    Determines what device is available (e.g. running on colab gpu instance should return cuda whereas local run might return cpu)

    Args:
        None
    
    Returns:
        str: device string
    """
    # Call methods on torch package to see what device is available
    # Print statements for easier visibility on what is being used
    if torch.cuda.is_available():
        device = "cuda"
        print("Using CUDA")
    elif torch.backends.mps.is_available():
        device = "mps"
        print("Using MPS")
    elif torch.backends.opencl.is_available():
        device = "opencl"
        print("Using OpenCL")
    else:
        device = "cpu"
        print("Using CPU")
    return device

In [ ]:
# These were the only markdown separators tested which appeared to split the documents reasonably well
# This is another area where additional work could be completed to test if utilising different separators would lead to better performance for RAG
MARKDOWN_SEPARATORS = [
    "\n#{1,6} ",
    "```\n",
    "\n\\*\\*\\*+\n",
    "\n---+\n",
    "\n___+\n",
    "\n\n",
    "\n",
    " ",
    "",
]

def split_documents(
    chunk_size: int,
    knowledge_base: List[LangchainDocument],
    tokenizer_name: Optional[str] = MEDIUM_TOKENISER_MODEL_NAME,
) -> List[LangchainDocument]:
    """
    Split documents into chunks of maximum size `chunk_size` tokens and return a list of documents.

    Args:
        chunk_size (int): determines what size each chunk should be
        knowledge_base (List[LangchainDocument]): list of langchain documents that are to be split into chunks
        tokenizer_name (Optional[str]): What tokenizer model should be used
    
    Returns:
        List[LangchainDocument]: Returns chunked documents
    """

    # Use the RecursiveCharacterTextSplitter to split the data into chunks based on parameter values
    # Note the overlap is set to 10% to minimise chance of context loss between chunks
    text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
        AutoTokenizer.from_pretrained(tokenizer_name),
        chunk_size=chunk_size,
        chunk_overlap=int(chunk_size / 10),
        add_start_index=True,
        strip_whitespace=True,
        separators=MARKDOWN_SEPARATORS,
    )

    docs_processed = []
    for doc in knowledge_base:
        # Loop over documents and append chunk to docs_processed variable
        docs_processed += text_splitter.split_documents([doc])

    # Remove duplicates
    unique_texts = {}
    docs_processed_unique = []

    # Check to ensure that no duplicates are included
    for doc in docs_processed:
        if doc.page_content not in unique_texts:
            unique_texts[doc.page_content] = True
            docs_processed_unique.append(doc)

    return docs_processed_unique

In [ ]:
def generate_faiss_embeddings_vector_store(documents: List[LangchainDocument], embedding_model: Embeddings, batch_size: int = 32) -> FAISS:
  """
  Generate embedding store based on provided documents with provided embedding model

  Args:
    documents (List[LangchainDocument]): knowledge base which is to be used for vector store
    embedding_model (Embeddings): model used to generate embeddings for documents
    batch_size (batch_size): how many documents should embeddings be generated for in each iteration
  
  Returns:
    FAISS: vector store for the provided documents
  """

  # First, initialise to None
  faiss_index = None

  # Loop over all documents, increment based on batch size variable value
  for i in tqdm(range(0, len(documents), batch_size)):
    # Extract documents in current batch
    documents_current_batch = documents[i:i+batch_size]

    # Extract text from documents in current batch
    documents_current_batch_extracted_text = [doc.page_content for doc in documents_current_batch]

    # Use the embedding model to generate embeddings for the current batch text
    current_batch_embeddings = embedding_model.embed_documents(documents_current_batch_extracted_text)

    # Create list of tuples linking the raw text with the current batch embeddings generated
    current_batch_embeddings_pairs = list(zip(documents_current_batch_extracted_text, current_batch_embeddings))

    # Create FAISS index for current batch embeddings by passing embeddings and embedding model
    batch_faiss = FAISS.from_embeddings(text_embeddings=current_batch_embeddings_pairs, embedding=embedding_model)

    # If first batch, set faiss index to FAISS index, otherwise merge to ensure previous batch values are not overwritten
    if faiss_index is None:
      faiss_index = batch_faiss
    else:
      faiss_index.merge_from(batch_faiss)

  # Once all batches have been processed, return the full index store
  return faiss_index

In [ ]:
def check_file_exists(file_path: str) -> bool:
  """
  This function checks if provided file path exists

  Args:
    file_path (str): file path which should be checked for file existence
  
  Returns:
    bool: whether the file exists or not
  """

  # Uses os package isfile method to determine if a file exists at the provided file path
  # If so, return True, else return False
  if os.path.isfile(file_path):
    return True

  return False

In [ ]:
def write_json_file(output_file_path: str, content: Any) -> bool:
  """
  Writes content to json file at provided output path

  Args:
    output_file_path (str): File path at which file should be generated
    content (Any): The content to be written within the file
  
  Returns:
    bool: indicates if writing json was successful or not
  """

  # Get directory name from output file path parameter value
  output_directory = os.path.dirname(output_file_path)

  # If the directory does not exist, create it
  if output_directory and not os.path.exists(output_directory):
    os.makedirs(output_directory)

  # Try writing content to the file - return True if all successful, otherwise catch the error, log and return False
  try:
    with open(output_file_path, "w") as output_file:
      json.dump(content, output_file, indent=4)

    print("Successfully saved content to {}".format(output_file_path))

    return True
  except(OSError, IOError) as e:
    print("Error saving content to {}".format(output_file_path))
    return False

In [ ]:
def read_json_file(input_file_path: str, default: Any = []) -> Any:
  """
  Reads content from json file at provided input file path

  Args:
    input_file_path (str): the file path where the target file resides
    default (Any): the default structure of the expected file (to be returned in case of error to avoid consuming errors)
  
  Returns:
    Any: the content read from the file
  """

  # Call utility function to check if a file actually exists at the provided file path parameter
  # If it does not exist, return the default data structure
  if not check_file_exists(input_file_path):
    return default

  # Try open the file and read the contents
  # If successful, return the contents. If not successful, return the default data structure
  try:
    with open(input_file_path, "r", encoding="utf-8") as input_file:
      content = json.load(input_file)

    print("Successfully loaded content from {} file".format(input_file_path))
    return content

  except (OSError, IOError, json.JSONDecodeError) as e:
    print("Error reading from file path: {}".format(input_file_path))
    return default

In [ ]:
def generate_json_file_name(prompt_type: str, model_name: str, model_type: str, temperature: float, top_k: int, top_p: float, repetition_penalty: float, no_repeat_n_grams: float, max_new_tokens: int, instance_name: str, output_directory: str) -> str:
  """
  Generate unique file name to be used for model experiment files

  Args:
    prompt_type (str): experiment prompt type (zero-shot, few-shot, instruction, persona, chain-of-thought, tree-of-thought)
    model_name (str): experiment model
    model_type (str): experiment model type (rag / llm)
    temperature (str): value for temperature hyperparameter
    top_k (int): top k hyperparameter value
    top_p (float): top p hyperparameter value
    repetition_penalty (float): rep penalty hyperparameter value
    no_repeat_n_grams (float): no repeat n grams hyperparameter value
    max_new_tokens (int): hyperparameter value for max new tokens
    instance_name (str): Google Colaboratory instance name used in experiment
    output_directory (str): output directory where results are to be written
  
  Returns:
    str: the unique file name for the model experiment results
  """

  # Replace spaces and / in model name and replace with _ for safe writing for files
  safe_model_name = model_name.replace("/", "_").replace(" ", "_")

  # Likewise for prompt and instance, remove any spaces and / characters
  safe_prompt_name = prompt_type.replace(" ", "_")
  safe_instance_name = instance_name.replace("/", "_").replace(" ", "_")

  # Generate timestamp to ensure uniqueness for file name
  timestamp = datetime.now().strftime("%d-%m-%Y_%H-%M-%S")

  # Substitute parameters into file path structure
  file_path = "{}/golden_results_{}_{}_{}_{}_temp{}_topk{}_topP{}_repPenalty{}_noRepeatNGrams{}_max{}_{}.json".format(output_directory, safe_prompt_name, safe_model_name, safe_instance_name, model_type, temperature, top_k, top_p, repetition_penalty, no_repeat_n_grams, max_new_tokens, timestamp)

  # Return generated file path
  return file_path

In [ ]:
def get_instance_name() -> str:
  """
  Get current Google colab instance name

  Args:
    None
  
  Returns:
    str: the instance name for google colab, otherwise if not able to extract, default to CPU
  """

  # Use torch method to get current instance name if available otherwise default to CPU
  return torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU"

In [ ]:
def generate_rouge_scores(ground_truth: str, generated_answer: str) -> Tuple[float, float, float]:
  """
  Generate rouge scores by providing ground truth and generated answer to rouge score package

  Args:
    ground_truth (str): Ground truth which should be used as baseline for rouge score generation
    generated_answer (str): model generated output
  
  Returns:
    Tuple[float, float, float]: rouge1, rouge2, rougel scores
  """

  # Call RougeScorer method initialise class with target rouge metrics
  scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)

  # Actually generate scores
  rouge_scores = scorer.score(ground_truth, generated_answer)

  # Extract scores
  rouge_1_score = rouge_scores["rouge1"].fmeasure
  rouge_2_score = rouge_scores["rouge2"].fmeasure
  rouge_l_score = rouge_scores["rougeL"].fmeasure

  # Return scores
  return rouge_1_score, rouge_2_score, rouge_l_score

In [ ]:
def get_file_paths_matching_regex(directory: str, regex_pattern: Union[str, Pattern]) -> List[str]:
  """
  Get files within provided directory path that match provided regex pattern

  Args:
    directory (str): target directory which to search for matched files
    regex_pattern (Union[str, Pattern]): regex used to match docs
  
  Returns:
    List[str]: return list of matched file paths
  """

  # Initialise empty dict
  unique_files_dict = {}

  # Traverse directory files
  # If match found for file with regex, set file name as key and full file path as value in dict
  for root, dirs, files in os.walk(directory):
    for file_name in files:
      if re.search(regex_pattern, file_name):
        full_path = os.path.join(root, file_name)

        unique_files_dict[file_name] = full_path

  # Once all have been looped over, generate list of full path values by extracting values from dict
  unique_files = list(unique_files_dict.values())

  # Return list of file paths that match regex pattern within provided directory
  return unique_files

In [ ]:
def plot_rouge_score_comparison(first_model_results: List[Dict[str, float]], first_model_name: str, second_model_results: List[Dict[str, float]], second_model_name: str) -> None:
  """
  Use matplotlib to plot comparison between two models for generated rouge scores

  Args:
    first_model_results (List[Dict[str, float]]): results for first model to be plotted
    first_model_name (str): first model name
    second_model_results (List[Dict[str, float]]): results for second model to be plotted
    second_model_name (str): second model name
  
  Returns:
    None
  """

  # Extract properties from results for both models
  rouge_one_scores_model_one = [i["rouge1_score"] for i in first_model_results]
  rouge_one_scores_model_two = [i["rouge1_score"] for i in second_model_results]

  rouge_two_scores_model_one = [i["rouge2_score"] for i in first_model_results]
  rouge_two_scores_model_two = [i["rouge2_score"] for i in second_model_results]

  rouge_l_scores_model_one = [i["rougeL_score"] for i in first_model_results]
  rouge_l_scores_model_two = [i["rougeL_score"] for i in second_model_results]

  # Initialise graph
  plt.figure(figsize=(12,4))

  # Use subplots to plot side by side

  # First graph for rouge 1
  plt.subplot(1,3,1)
  plt.scatter(rouge_one_scores_model_one, rouge_one_scores_model_two, color="blue", alpha=0.6, label="ROUGE-1")
  plt.plot([0,1], [0,1], "--", color="gray")
  plt.xlabel("{} ROUGE-1".format(first_model_name))
  plt.ylabel("{} ROUGE-1".format(second_model_name))
  plt.title("ROUGE-1 Comparison")
  plt.legend()

  # Second graph for rouge 2
  plt.subplot(1,3,2)
  plt.scatter(rouge_two_scores_model_one, rouge_two_scores_model_two, color="green", alpha=0.6, label="ROUGE-2")
  plt.plot([0,1], [0,1], "--", color="gray")
  plt.xlabel("{} ROUGE-2".format(first_model_name))
  plt.ylabel("{} ROUGE-2".format(second_model_name))
  plt.title("ROUGE-2 Comparison")
  plt.legend()

  # Third graph for rouge L
  plt.subplot(1,3,3)
  plt.scatter(rouge_l_scores_model_one, rouge_l_scores_model_two, color="red", alpha=0.6, label="ROUGE-L")
  plt.plot([0,1], [0,1], "--", color="gray")
  plt.xlabel("{} ROUGE-L".format(first_model_name))
  plt.ylabel("{} ROUGE-L".format(second_model_name))
  plt.title("ROUGE-L Comparison")
  plt.legend()

  plt.tight_layout()
  plt.show()

In [ ]:
def plot_score_comparison_recall_precision_f1(first_model_results: List[Dict[str, float]], first_model_name: str, second_model_results: List[Dict[str, float]], second_model_name: str) -> None:
  """
  Use matplotlib to plot comparison between two models for generated recall, precision, f1 scores

  Args:
    first_model_results (List[Dict[str, float]]): results for first model to be plotted
    first_model_name (str): first model name
    second_model_results (List[Dict[str, float]]): results for second model to be plotted
    second_model_name (str): second model name
  
  Returns:
    None
  """
  
  # Extract precision, recall, f1 scores from results to be plotted
  bert_precision_scores_model_one = [i["bert_precision"] for i in first_model_results]
  bert_precision_scores_model_two = [i["bert_precision"] for i in second_model_results]

  bert_recall_scores_model_one = [i["bert_recall"] for i in first_model_results]
  bert_recall_scores_model_two = [i["bert_recall"] for i in second_model_results]

  bert_f1_scores_model_one = [i["bert_f1"] for i in first_model_results]
  bert_f1_scores_model_two = [i["bert_f1"] for i in second_model_results]

  # Initialise graph
  plt.figure(figsize=(12,4))

  # On first subplot, plot precision
  plt.subplot(1,3,1)
  plt.scatter(bert_precision_scores_model_one, bert_precision_scores_model_two, color="blue", alpha=0.6, label="BERT Precision")
  plt.plot([0,1], [0,1], "--", color="gray")
  plt.xlabel("{} BERT Precision".format(first_model_name))
  plt.ylabel("{} BERT Precision".format(second_model_name))
  plt.title("BERT Precision Comparison")
  plt.legend()

  # On second subplot, plot recall
  plt.subplot(1,3,2)
  plt.scatter(bert_recall_scores_model_one, bert_recall_scores_model_two, color="green", alpha=0.6, label="BERT Recall")
  plt.plot([0,1], [0,1], "--", color="gray")
  plt.xlabel("{} BERT Recall".format(first_model_name))
  plt.ylabel("{} BERT Recall".format(second_model_name))
  plt.title("BERT Recall Comparison")
  plt.legend()

  # On third subplot, plot F1
  plt.subplot(1,3,3)
  plt.scatter(bert_f1_scores_model_one, bert_f1_scores_model_two, color="red", alpha=0.6, label="BERT F1")
  plt.plot([0,1], [0,1], "--", color="gray")
  plt.xlabel("{} BERT F1".format(first_model_name))
  plt.ylabel("{} BERT F1".format(second_model_name))
  plt.title("BERT F1 Comparison")
  plt.legend()

  plt.tight_layout()
  plt.show()

In [ ]:
def delete_model(model_variable: Any) -> None:
  """
  Free up GPU and CPU resources when no longer needed for model by deleting reference and calling garbage collector and emptying torch cache
  """
  
  model_variable.pipeline.model.to("cpu")
  del model_variable
  gc.collect()
  torch.cuda.empty_cache()
  torch.cuda.ipc_collect()

In [ ]:
def get_prompt_by_type(prompt_type: str, prompt_dict: Dict = {}) -> str:
  """
  Utility function to easily extract prompt template based on prompt type parameter value

  Args:
    prompt_type (str): the target key for which the value should be returned
    prompt_dict (Dict): the dictionary that stores prompt types as keys and prompt templates as values
  """

  # Sanitise input for prompt type
  prompt_type = prompt_type.strip().lower()

  # Try return prompt type template based on type parameter value, otherwise throw error
  try:
    return prompt_dict[prompt_type]
  except KeyError:
    raise ValueError("Unknown prompt type: {}".format(prompt_type))

In [ ]:
def get_post_process_function_by_prompt_type(prompt_type: str, prompt_dict: Dict = {}) -> str:
  """
  Utility function to easily extract post process function based on prompt type parameter value

  Args:
    prompt_type (str): the target key for which the value should be returned
    prompt_dict (Dict): the dictionary that stores prompt types as keys and post-process functions as values
  """

  # Sanitise input for prompt type
  prompt_type = prompt_type.strip().lower()

  # Try return post process function based on prompt type parameter value, otherwise throw error
  try:
    return prompt_dict[prompt_type]
  except KeyError:
    raise ValueError("Unknown prompt type: {}".format(prompt_type))

In [ ]:
def retry_hugging_face_inference(func: Callable[..., Any], *args: Any, max_retries: int = 5, base_wait_period: int = 10, retry_on: Tuple[Type[BaseException, ...]]=(HTTPError,)) -> None:
  """
  Occasionally, hugging face inference timeouts were observed (if response time was very low, loops could lead to timeout issues)
  Utility function to add retry functionality to catch these errors and retry to improve stability

  Args:
    func (Callable[..., Any]): the function which should be retried in case of failure
    args (Any): the arguments passed to the function
    max_retries (int): number of retries before actually failing
    base_wait_period (int): wait period between retries
    retry_on (Tuple[Type[BaseException, ...]]): list of errors on which to retry
  
  Returns:
    None
  """

  # Initialise retries at 0
  retries = 0

  # If unsuccessful request, keep retrying until max retries value reached
  while retries < max_retries:
    # Try call the function, in case of failure code in provided parameter, increment and retry after wait period
    # Otherwise, print output informing user of failure in case of other exceptions or if max retries is exceeded
    try:
      return func(*args)
    except retry_on as e:
      status_code = getattr(e.response, "status_code", None)

      if status_code == 429:
        wait = base_wait_period * (2 * retries)
        print("Too many requests, retrying in {} seconds...".format(wait))
        time.sleep(wait)
        retries += 1
      else:
        raise
    except Exception as e:
      print("Error {}: {}".format(type(e).__name__, e))
      raise
  raise Exception("max retries exceeded")

In [ ]:
def delete_files_in_directory(directory_path: str) -> None:
  """
  Delete files in a provided directory

  Args:
    directory_path (str): the directory path at which the files should be deleted

  Returns:
    None
  """

  # If directory path does not exist, inform user of error
  if not os.path.exists(directory_path):
    print("Directory {} does not exist".format(directory_path))

  # Initialise file count (will be used to count number of deleted files)
  file_count = 0

  # Loop over files in directory, remove files and increment file count as each one is deleted
  for file_name in os.listdir(directory_path):
    file_path = os.path.join(directory_path, file_name)

    if os.path.isfile(file_path):
      os.remove(file_path)
      file_count += 1

  # Once complete, inform user how many files were deleted from the directory
  print("Deleted {} files from {} directory".format(file_count, directory_path))


## Clone Repo with Scraped Data

In [ ]:
# Citizen information data was scraped over a number of runs on my local machine
# Therefore, need to clone down the repo to get access to the data
!git clone https://github.com/JohnDennehy101/webScraperCitizensInformation.git

Cloning into 'webScraperCitizensInformation'...
remote: Enumerating objects: 41164, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 41164 (delta 27), reused 37 (delta 21), pack-reused 41111 (from 2)
Receiving objects: 100% (41164/41164), 59.75 MiB | 14.49 MiB/s, done.
Resolving deltas: 100% (6051/6051), done.
Updating files: 100% (43422/43422), done.


## Extract Documents (and filter out Gaeilge documents)

In [ ]:
# Directory path within the repo where target documents are available
directory_path = "/content/webScraperCitizensInformation/src/data/markdown"

# Use the directory loader to load the files, note .md used as files are in markdown format
loader = DirectoryLoader(
    path=directory_path,
    glob="*.md",
    loader_cls=UnstructuredMarkdownLoader,
    recursive=True
)

# Load documents
documents = loader.load()

print(len(documents))

# Filter out documents that start with 'ga' which signifies that they are in Irish (which is not well supported by LLMs)
# Therefore, decision made to only use English language documents for the experiments
filtered_documents = [
    doc for doc in documents if not os.path.basename(doc.metadata['source']).startswith('ga_')
]

if documents:
    print("Contents of the first file:")
    print(documents[0].page_content)
else:
    print("No documents found.")

2840
Contents of the first file:
English | Gaeilge

You are here: Home > Housing > Planning permission > Development plans

Development plans

What is a development plan?

Do national plans influence local development plans?

Can I give feedback about a development plan?

Can a development plan influence my application for planning permission?

More information on development plans

What is a development plan?

A development plan describes how your local authority will develop and use particular areas in your local authority area, for example:

Residential areas

Commercial areas

Industrial areas

Recreational areas

Agricultural areas

It also sets out the development objectives for your local authority area, such as plans to improve roads and local amenities.

A development plan is made up of a written statement and series of maps.

How long does a development plan last?

A development plan lasts for 6 years. Local authorities must begin work on a new development plan 4 years after 

In [ ]:
print(len(filtered_documents))

if filtered_documents:
    print("Contents of the first file:")
    print(filtered_documents[0].page_content)
else:
    print("No documents found.")

1924
Contents of the first file:
English | Gaeilge

You are here: Home > Moving Country > Moving to Ireland > Rights of residence in Ireland > Residence rights of family members

Residence rights of family members

Introduction

Family members of Irish citizens

Family members of UK citizens

Family members of EU, EEA and Swisscitizens

People who have internationalprotection status

Other Non-EEA citizens

Dependent elderly relatives

Further information and contacts

Introduction

If you move to Ireland you may be able to bring your Non-EEA family members to live with you. Your right to have your family join you in Ireland depends on:

Your residence status in Ireland

Your family relationship with the person

In some cases, you have a legal right to be joined by your family. But in many situations, you must show that you have the income to support them and satisfy eligibility criteria. The information on this page is mostly about spouses, partners and dependent children.

Family mem

In [ ]:
# Extract page content for each English langugage document and store as list of LangchainDocuments
RAW_KNOWLEDGE_BASE = [
    LangchainDocument(page_content=doc.page_content) for doc in tqdm(filtered_documents)
]

100%|██████████| 1924/1924 [00:00<00:00, 246603.13it/s]


In [ ]:
# Split the documents into chunks using the utility split_documents function
docs_processed = split_documents(
    CHUNK_SIZE,
    RAW_KNOWLEDGE_BASE,
    tokenizer_name=MEDIUM_TOKENISER_MODEL_NAME
)

In [ ]:
avg_doc_length = lambda docs: sum([len(doc.page_content) for doc in docs])//len(docs)

In [ ]:
avg_char_before_split = avg_doc_length(RAW_KNOWLEDGE_BASE)
avg_char_after_split = avg_doc_length(docs_processed)

In [ ]:
# Print out documents for values before split
print("Before split, there were {} documents loaded, with average characters equal to {}.".format(len(RAW_KNOWLEDGE_BASE), avg_char_before_split))

Before split, there were 1924 documents loaded, with average characters equal to 6807.


In [ ]:
# Print out documents for values after chunking to show that there are more chunks with fewer characters in each
print("After split, there were {} documents (chunks), with average characters equal to {} (average chunk length).".format(len(docs_processed), avg_char_after_split))

After split, there were 3632 documents (chunks), with average characters equal to 3373 (average chunk length).


In [ ]:
# Get device type by calling utility function
device = get_device()

# Initialise embedding model
huggingface_embeddings_medium_model = HuggingFaceBgeEmbeddings(
    model_name=MEDIUM_TOKENISER_MODEL_NAME,
    model_kwargs={"device": device,},
    encode_kwargs={"normalize_embeddings": True, "multi_process": False}
)

Using CUDA


/tmp/ipython-input-28-3755219830.py:3: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  huggingface_embeddings_medium_model = HuggingFaceBgeEmbeddings(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnin

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/284 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/80.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/370 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

In [ ]:
# For vector store, initialise to None initially
vectorstore_medium_model = None

# Vector store generation took a significant amount of time, therefore model files were saved for re-use between model runs to speed up execution
# First, check if the path for the embedding files if present (if so try and use it)
if os.path.isdir(VECTOR_DB_PATH):
  print("Vector db found locally, loading")
  vectorstore_medium_model = FAISS.load_local(folder_path=VECTOR_DB_PATH, embeddings=huggingface_embeddings_medium_model, allow_dangerous_deserialization=True)
# Otherwise, construct the vector db by calling the utility generate_faiss_embeddings_vector_store function
else:
  print("Not found, building vector db")
  vectorstore_medium_model = generate_faiss_embeddings_vector_store(docs_processed, huggingface_embeddings_medium_model)
  vectorstore_medium_model.save_local("faiss_vector_store_{}.index".format(MEDIUM_TOKENISER_MODEL_NAME.replace("/", "_")))

Vector db found locally, loading


In [ ]:

# Initialise retriever
retriever = vectorstore_medium_model.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [ ]:
# These variables were used to ensure that any tweaks here were constant throughout the file (easier when running multiple experiments)
LLM_IN_USE_NAME = LARGE_LLM_MODEL_NAME
PROMPT_TYPE_IN_USE = FEW_SHOT_PROMPT_TYPE
RUN_ALL_PROMPTS = False

RE_RANKER = "colbert-ir/colbertv2.0"
NUMBER_RETRIEVED_DOCS = 20
NUMBER_FINAL_DOCS = 5


TEMPERATURE = 0
TOP_K = 0
TOP_P = 0
REPETITION_PENALTY = 0
NO_REPEAT_N_GRAMS = 0
MAX_TOKENS = 500

In [ ]:
# Experiments only ran:

##### Zero Shot

# LARGE MODEL

# {}
# {"temperature": 1e-5}
# {"temperature": 1e-5, "max_new_tokens": 500}
# {"temperature": 1e-5, "max_new_tokens": 750}

# {"top_k": 10, "do_sample": True}
# {"top_k": 30, "do_sample": True}
# {"top_k": 50, "do_sample": True}

# {"top_p": 0.7, "do_sample": True}
# {"top_p": 0.8, "do_sample": True}
# {"top_p": 0.9, "do_sample": True}

# {"repetition_penalty": 1.1, "do_sample": True}
# {"repetition_penalty": 1.2, "do_sample": True}
# {"repetition_penalty": 1.3, "do_sample": True}

# {"no_repeat_ngram_size": 2, "do_sample": True}

# {"temperature": 0.5, "do_sample": True}

# {"max_new_tokens": 100}
# {"max_new_tokens": 150}
# {"max_new_tokens": 200}

# {"temperature": 1e-5, "max_new_tokens": 100}
# {"temperature": 1e-5, "max_new_tokens": 150}
# {"temperature": 1e-5, "max_new_tokens": 200}

# {"temperature": 0.5, "do_sample": True, "max_new_tokens": 150}

# {"repetition_penalty": 1.1, "do_sample": True, "max_new_tokens": 150}
# {"repetition_penalty": 1.2, "do_sample": True, "max_new_tokens": 150}
# {"repetition_penalty": 1.3, "do_sample": True, "max_new_tokens": 150}

# {"top_k": 10, "do_sample": True, "max_new_tokens": 150}
# {"top_k": 30, "do_sample": True, "max_new_tokens": 150}
# {"top_k": 50, "do_sample": True, "max_new_tokens": 150}

# {"top_p": 0.7, "do_sample": True, "max_new_tokens": 150}
# {"top_p": 0.8, "do_sample": True, "max_new_tokens": 150}
# {"top_p": 0.9, "do_sample": True, "max_new_tokens": 150}

# {"no_repeat_ngram_size": 2, "do_sample": True, "max_new_tokens": 150}

# {"max_new_tokens": 500}
# {"max_new_tokens": 750}
# {"max_new_tokens": 1000}
# {"max_new_tokens": 1500}


##### Basic Instruction Prompt

# LARGE MODEL

# {}

# {"temperature": 1e-5}
# {"temperature": 1e-5, "max_new_tokens": 500}
# {"temperature": 1e-5, "max_new_tokens": 750}

# {"top_k": 10, "do_sample": True}
# {"top_k": 30, "do_sample": True}
# {"top_k": 50, "do_sample": True}

# {"top_p": 0.7, "do_sample": True}
# {"top_p": 0.8, "do_sample": True}
# {"top_p": 0.9, "do_sample": True}

# {"repetition_penalty": 1.1, "do_sample": True}
# {"repetition_penalty": 1.2, "do_sample": True}
# {"repetition_penalty": 1.3, "do_sample": True}

# {"no_repeat_ngram_size": 2, "do_sample": True}

# {"temperature": 0.5, "do_sample": True}

# {"temperature": 0.5, "do_sample": True, "max_new_tokens": 150}
# {"temperature": 0, "do_sample": False, "max_new_tokens": 150}
# {"temperature": 1.0, "do_sample": True, "max_new_tokens": 150}

# {"top_k": 10, "do_sample": True, "max_new_tokens": 150}
# {"top_k": 30, "do_sample": True, "max_new_tokens": 150}
# {"top_k": 50, "do_sample": True, "max_new_tokens": 150}

# {"top_p": 0.7, "do_sample": True, "max_new_tokens": 150}
# {"top_p": 0.8, "do_sample": True, "max_new_tokens": 150}
# {"top_p": 0.9, "do_sample": True, "max_new_tokens": 150}

# {"repetition_penalty": 1.1, "do_sample": True, "max_new_tokens": 150}
# {"repetition_penalty": 1.2, "do_sample": True, "max_new_tokens": 150}
# {"repetition_penalty": 1.3, "do_sample": True, "max_new_tokens": 150}

# {"no_repeat_ngram_size": 2, "do_sample": True, "max_new_tokens": 150}

# {"temperature": 0, "max_new_tokens": 100}
# {"temperature": 0, "max_new_tokens": 200}

# {"max_new_tokens": 100}
# {"max_new_tokens": 200}

# {"max_new_tokens": 500}
# {"max_new_tokens": 750}
# {"max_new_tokens": 1000}
# {"max_new_tokens": 1500}

##### Few shot prompt

# LARGE MODEL

# {}

# {"temperature": 1e-5}
# {"temperature": 1e-5, "max_new_tokens": 500}
# {"temperature": 1e-5, "max_new_tokens": 750}

# {"top_k": 10, "do_sample": True}
# {"top_k": 30, "do_sample": True}
# {"top_k": 50, "do_sample": True}

# {"top_p": 0.7, "do_sample": True}
# {"top_p": 0.8, "do_sample": True}
# {"top_p": 0.9, "do_sample": True}

# {"repetition_penalty": 1.1, "do_sample": True}
# {"repetition_penalty": 1.2, "do_sample": True}
# {"repetition_penalty": 1.3, "do_sample": True}

# {"no_repeat_ngram_size": 2, "do_sample": True}

# {"temperature": 0.5, "do_sample": True}

# {"temperature": 0.5, "do_sample": True, "max_new_tokens": 150}

# {"repetition_penalty": 1.1, "do_sample": True, "max_new_tokens": 150}
# {"repetition_penalty": 1.2, "do_sample": True, "max_new_tokens": 150}
# {"repetition_penalty": 1.3, "do_sample": True, "max_new_tokens": 150}

# {"top_k": 10, "do_sample": True, "max_new_tokens": 150}
# {"top_k": 30, "do_sample": True, "max_new_tokens": 150}
# {"top_k": 50, "do_sample": True, "max_new_tokens": 150}

# {"top_p": 0.7, "do_sample": True, "max_new_tokens": 150}
# {"top_p": 0.8, "do_sample": True, "max_new_tokens": 150}
# {"top_p": 0.9, "do_sample": True, "max_new_tokens": 150}

# {"no_repeat_ngram_size": 2, "do_sample": True, "max_new_tokens": 150}

# {"max_new_tokens": 500}
# {"max_new_tokens": 750}
# {"max_new_tokens": 1000}
# {"max_new_tokens": 1500}
# {"max_new_tokens": 2000}

##### Chain of thought prompt

# LARGE MODEL

# {}

# {"temperature": 1e-5}
# {"temperature": 1e-5, "max_new_tokens": 500}
# {"temperature": 1e-5, "max_new_tokens": 750}

# {"top_k": 10, "do_sample": True}
# {"top_k": 30, "do_sample": True}
# {"top_k": 50, "do_sample": True}

# {"top_p": 0.7, "do_sample": True}
# {"top_p": 0.8, "do_sample": True}
# {"top_p": 0.9, "do_sample": True}

# {"repetition_penalty": 1.1, "do_sample": True}
# {"repetition_penalty": 1.2, "do_sample": True}
# {"repetition_penalty": 1.3, "do_sample": True}

# {"no_repeat_ngram_size": 2, "do_sample": True}

# {"temperature": 0.5, "do_sample": True}

# {"temperature": 0.5, "do_sample": True, "max_new_tokens": 150}

# {"repetition_penalty": 1.1, "do_sample": True, "max_new_tokens": 150}
# {"repetition_penalty": 1.2, "do_sample": True, "max_new_tokens": 150}
# {"repetition_penalty": 1.3, "do_sample": True, "max_new_tokens": 150}

# {"top_k": 10, "do_sample": True, "max_new_tokens": 150}
# {"top_k": 30, "do_sample": True, "max_new_tokens": 150}
# {"top_k": 50, "do_sample": True, "max_new_tokens": 150}

# {"top_p": 0.7, "do_sample": True, "max_new_tokens": 150}
# {"top_p": 0.8, "do_sample": True, "max_new_tokens": 150}
# {"top_p": 0.9, "do_sample": True, "max_new_tokens": 150}

# {"no_repeat_ngram_size": 2, "do_sample": True, "max_new_tokens": 150}

# {"max_new_tokens": 500}
# {"max_new_tokens": 750}
# {"max_new_tokens": 1000}
# {"max_new_tokens": 1500}
# {"max_new_tokens": 2000}

##### Tree of thought prompt

# LARGE MODEL

# {}

# {"temperature": 1e-5}
# {"temperature": 1e-5, "max_new_tokens": 500}
# {"temperature": 1e-5, "max_new_tokens": 750}

# {"top_k": 10, "do_sample": True}
# {"top_k": 30, "do_sample": True}
# {"top_k": 50, "do_sample": True}

# {"top_p": 0.7, "do_sample": True}
# {"top_p": 0.8, "do_sample": True}
# {"top_p": 0.9, "do_sample": True}

# {"repetition_penalty": 1.1, "do_sample": True}
# {"repetition_penalty": 1.2, "do_sample": True}
# {"repetition_penalty": 1.3, "do_sample": True}

# {"no_repeat_ngram_size": 2, "do_sample": True}

# {"temperature": 0.5, "do_sample": True}
# {"temperature": 0.5, "do_sample": True, "max_new_tokens": 150}

# {"repetition_penalty": 1.1, "do_sample": True, "max_new_tokens": 150}
# {"repetition_penalty": 1.2, "do_sample": True, "max_new_tokens": 150}
# {"repetition_penalty": 1.3, "do_sample": True, "max_new_tokens": 150}

# {"top_k": 10, "do_sample": True, "max_new_tokens": 150}
# {"top_k": 30, "do_sample": True, "max_new_tokens": 150}
# {"top_k": 50, "do_sample": True, "max_new_tokens": 150}

# {"top_p": 0.7, "do_sample": True, "max_new_tokens": 150}
# {"top_p": 0.8, "do_sample": True, "max_new_tokens": 150}
# {"top_p": 0.9, "do_sample": True, "max_new_tokens": 150}

# {"no_repeat_ngram_size": 2, "do_sample": True, "max_new_tokens": 150}

# {"max_new_tokens": 500}
# {"max_new_tokens": 750}
# {"max_new_tokens": 1000}
# {"max_new_tokens": 1500}
# {"max_new_tokens": 2000}

##### Persona prompt

# LARGE MODEL

# {}

# {"temperature": 1e-5}
# {"temperature": 1e-5, "max_new_tokens": 500}
# {"temperature": 1e-5, "max_new_tokens": 750}

# {"top_k": 10, "do_sample": True}
# {"top_k": 30, "do_sample": True}
# {"top_k": 50, "do_sample": True}

# {"top_p": 0.7, "do_sample": True}
# {"top_p": 0.8, "do_sample": True}
# {"top_p": 0.9, "do_sample": True}

# {"repetition_penalty": 1.1, "do_sample": True}
# {"repetition_penalty": 1.2, "do_sample": True}
# {"repetition_penalty": 1.3, "do_sample": True}

# {"no_repeat_ngram_size": 2, "do_sample": True}

# {"temperature": 0.5, "do_sample": True}

# {"temperature": 0.5, "do_sample": True, "max_new_tokens": 150}

# {"repetition_penalty": 1.1, "do_sample": True, "max_new_tokens": 150}
# {"repetition_penalty": 1.2, "do_sample": True, "max_new_tokens": 150}
# {"repetition_penalty": 1.3, "do_sample": True, "max_new_tokens": 150}

# {"top_k": 10, "do_sample": True, "max_new_tokens": 150}
# {"top_k": 30, "do_sample": True, "max_new_tokens": 150}
# {"top_k": 50, "do_sample": True, "max_new_tokens": 150}

# {"top_p": 0.7, "do_sample": True, "max_new_tokens": 150}
# {"top_p": 0.8, "do_sample": True, "max_new_tokens": 150}
# {"top_p": 0.9, "do_sample": True, "max_new_tokens": 150}

# {"no_repeat_ngram_size": 2, "do_sample": True, "max_new_tokens": 150}

# {"max_new_tokens": 500}
# {"max_new_tokens": 750}
# {"max_new_tokens": 1000}
# {"max_new_tokens": 1500}
# {"max_new_tokens": 2000}

# **************************

##### Zero shot

# MEDIUM MODEL

# {}

# {"temperature": 1e-5}
# {"temperature": 1e-5, "max_new_tokens": 500}
# {"temperature": 1e-5, "max_new_tokens": 750}

# {"top_k": 10, "do_sample": True}
# {"top_k": 30, "do_sample": True}
# {"top_k": 50, "do_sample": True}

# {"top_p": 0.7, "do_sample": True}
# {"top_p": 0.8, "do_sample": True}
# {"top_p": 0.9, "do_sample": True}

# {"repetition_penalty": 1.1, "do_sample": True}
# {"repetition_penalty": 1.2, "do_sample": True}
# {"repetition_penalty": 1.3, "do_sample": True}

# {"no_repeat_ngram_size": 2, "do_sample": True}

# {"temperature": 0.5, "do_sample": True}

# {"max_new_tokens": 100}
# {"max_new_tokens": 150}
# {"max_new_tokens": 200}

# {"temperature": 1e-5, "max_new_tokens": 100}
# {"temperature": 1e-5, "max_new_tokens": 150}
# {"temperature": 1e-5, "max_new_tokens": 200}

# {"temperature": 0.5, "do_sample": True, "max_new_tokens": 150}

# {"repetition_penalty": 1.1, "do_sample": True, "max_new_tokens": 150}
# {"repetition_penalty": 1.2, "do_sample": True, "max_new_tokens": 150}
# {"repetition_penalty": 1.3, "do_sample": True, "max_new_tokens": 150}

# {"top_k": 10, "do_sample": True, "max_new_tokens": 150}
# {"top_k": 30, "do_sample": True, "max_new_tokens": 150}
# {"top_k": 50, "do_sample": True, "max_new_tokens": 150}

# {"top_p": 0.7, "do_sample": True, "max_new_tokens": 150}
# {"top_p": 0.8, "do_sample": True, "max_new_tokens": 150}
# {"top_p": 0.9, "do_sample": True, "max_new_tokens": 150}

# {"no_repeat_ngram_size": 2, "do_sample": True, "max_new_tokens": 150}

# {"max_new_tokens": 500}
# {"max_new_tokens": 750}
# {"max_new_tokens": 1000}
# {"max_new_tokens": 1500}
# {"max_new_tokens": 2000}

##### Basic Instruction Prompt

# MEDIUM MODEL

# {}

# {"temperature": 1e-5}
# {"temperature": 1e-5, "max_new_tokens": 500}
# {"temperature": 1e-5, "max_new_tokens": 750}

# {"top_k": 10, "do_sample": True}
# {"top_k": 30, "do_sample": True}
# {"top_k": 50, "do_sample": True}

# {"top_p": 0.7, "do_sample": True}
# {"top_p": 0.8, "do_sample": True}
# {"top_p": 0.9, "do_sample": True}

# {"repetition_penalty": 1.1, "do_sample": True}
# {"repetition_penalty": 1.2, "do_sample": True}
# {"repetition_penalty": 1.3, "do_sample": True}

# {"no_repeat_ngram_size": 2, "do_sample": True}

# {"temperature": 0.5, "do_sample": True}

# {"max_new_tokens": 100}
# {"max_new_tokens": 150}
# {"max_new_tokens": 200}

# {"temperature": 1e-5, "max_new_tokens": 100}
# {"temperature": 1e-5, "max_new_tokens": 150}
# {"temperature": 1e-5, "max_new_tokens": 200}

# {"temperature": 0.5, "do_sample": True, "max_new_tokens": 150}

# {"repetition_penalty": 1.1, "do_sample": True, "max_new_tokens": 150}
# {"repetition_penalty": 1.2, "do_sample": True, "max_new_tokens": 150}
# {"repetition_penalty": 1.3, "do_sample": True, "max_new_tokens": 150}

# {"top_k": 10, "do_sample": True, "max_new_tokens": 150}
# {"top_k": 30, "do_sample": True, "max_new_tokens": 150}
# {"top_k": 50, "do_sample": True, "max_new_tokens": 150}

# {"top_p": 0.7, "do_sample": True, "max_new_tokens": 150}
# {"top_p": 0.8, "do_sample": True, "max_new_tokens": 150}
# {"top_p": 0.9, "do_sample": True, "max_new_tokens": 150}

# {"no_repeat_ngram_size": 2, "do_sample": True, "max_new_tokens": 150}

# {"temperature": 0.5, "do_sample": True, "max_new_tokens": 150}

# {"repetition_penalty": 1.1, "do_sample": True, "max_new_tokens": 150}
# {"repetition_penalty": 1.2, "do_sample": True, "max_new_tokens": 150}
# {"repetition_penalty": 1.3, "do_sample": True, "max_new_tokens": 150}

# {"top_k": 10, "do_sample": True, "max_new_tokens": 150}
# {"top_k": 30, "do_sample": True, "max_new_tokens": 150}
# {"top_k": 50, "do_sample": True, "max_new_tokens": 150}

# {"top_p": 0.7, "do_sample": True, "max_new_tokens": 150}
# {"top_p": 0.8, "do_sample": True, "max_new_tokens": 150}
# {"top_p": 0.9, "do_sample": True, "max_new_tokens": 150}

# {"no_repeat_ngram_size": 2, "do_sample": True, "max_new_tokens": 150}

# {"max_new_tokens": 500}
# {"max_new_tokens": 750}
# {"max_new_tokens": 1000}
# {"max_new_tokens": 1500}
# {"max_new_tokens": 2000}

##### Few shot prompt

# MEDIUM MODEL

# {}

# {"temperature": 1e-5}
# {"temperature": 1e-5, "max_new_tokens": 500}
# {"temperature": 1e-5, "max_new_tokens": 750}

# {"top_k": 10, "do_sample": True}
# {"top_k": 30, "do_sample": True}
# {"top_k": 50, "do_sample": True}

# {"top_p": 0.7, "do_sample": True}
# {"top_p": 0.8, "do_sample": True}
# {"top_p": 0.9, "do_sample": True}

# {"repetition_penalty": 1.1, "do_sample": True}
# {"repetition_penalty": 1.2, "do_sample": True}
# {"repetition_penalty": 1.3, "do_sample": True}

# {"no_repeat_ngram_size": 2, "do_sample": True}

# {"temperature": 0.5, "do_sample": True}

# {"repetition_penalty": 1.1, "do_sample": True, "max_new_tokens": 150}
# {"repetition_penalty": 1.2, "do_sample": True, "max_new_tokens": 150}
# {"repetition_penalty": 1.3, "do_sample": True, "max_new_tokens": 150}

# {"top_k": 10, "do_sample": True, "max_new_tokens": 150}
# {"top_k": 30, "do_sample": True, "max_new_tokens": 150}
# {"top_k": 50, "do_sample": True, "max_new_tokens": 150}

# {"top_p": 0.7, "do_sample": True, "max_new_tokens": 150}
# {"top_p": 0.8, "do_sample": True, "max_new_tokens": 150}
# {"top_p": 0.9, "do_sample": True, "max_new_tokens": 150}

# {"no_repeat_ngram_size": 2, "do_sample": True, "max_new_tokens": 150}

# {"temperature": 1e-5, "max_new_tokens": 100}
# {"temperature": 1e-5, "max_new_tokens": 150}
# {"temperature": 1e-5, "max_new_tokens": 200}

# {"temperature": 0.5, "do_sample": True, "max_new_tokens": 150}

# {"temperature": 0.5, "do_sample": True, "max_new_tokens": 150}

# {"repetition_penalty": 1.1, "do_sample": True, "max_new_tokens": 150}
# {"repetition_penalty": 1.2, "do_sample": True, "max_new_tokens": 150}
# {"repetition_penalty": 1.3, "do_sample": True, "max_new_tokens": 150}

# {"top_k": 10, "do_sample": True, "max_new_tokens": 150}
# {"top_k": 30, "do_sample": True, "max_new_tokens": 150}
# {"top_k": 50, "do_sample": True, "max_new_tokens": 150}

# {"top_p": 0.7, "do_sample": True, "max_new_tokens": 150}
# {"top_p": 0.8, "do_sample": True, "max_new_tokens": 150}
# {"top_p": 0.9, "do_sample": True, "max_new_tokens": 150}

# {"no_repeat_ngram_size": 2, "do_sample": True, "max_new_tokens": 150}

# {"max_new_tokens": 500}
# {"max_new_tokens": 750}
# {"max_new_tokens": 1000}
# {"max_new_tokens": 1500}
# {"max_new_tokens": 2000}

##### Chain of thought prompt

# MEDIUM MODEL

# {}

# {"temperature": 1e-5}
# {"temperature": 1e-5, "max_new_tokens": 500}
# {"temperature": 1e-5, "max_new_tokens": 750}

# {"top_k": 10, "do_sample": True}
# {"top_k": 30, "do_sample": True}
# {"top_k": 50, "do_sample": True}

# {"top_p": 0.7, "do_sample": True}
# {"top_p": 0.8, "do_sample": True}
# {"top_p": 0.9, "do_sample": True}

# {"repetition_penalty": 1.1, "do_sample": True}
# {"repetition_penalty": 1.2, "do_sample": True}
# {"repetition_penalty": 1.3, "do_sample": True}

# {"no_repeat_ngram_size": 2, "do_sample": True}

# {"temperature": 0.5, "do_sample": True}

# {"repetition_penalty": 1.1, "do_sample": True, "max_new_tokens": 150}
# {"repetition_penalty": 1.2, "do_sample": True, "max_new_tokens": 150}
# {"repetition_penalty": 1.3, "do_sample": True, "max_new_tokens": 150}

# {"top_k": 10, "do_sample": True, "max_new_tokens": 150}
# {"top_k": 30, "do_sample": True, "max_new_tokens": 150}
# {"top_k": 50, "do_sample": True, "max_new_tokens": 150}

# {"top_p": 0.7, "do_sample": True, "max_new_tokens": 150}
# {"top_p": 0.8, "do_sample": True, "max_new_tokens": 150}
# {"top_p": 0.9, "do_sample": True, "max_new_tokens": 150}

# {"no_repeat_ngram_size": 2, "do_sample": True, "max_new_tokens": 150}

# {"temperature": 1e-5, "max_new_tokens": 100}
# {"temperature": 1e-5, "max_new_tokens": 150}
# {"temperature": 1e-5, "max_new_tokens": 200}

# {"temperature": 0.5, "do_sample": True, "max_new_tokens": 150}

# {"temperature": 0.5, "do_sample": True, "max_new_tokens": 150}

# {"repetition_penalty": 1.1, "do_sample": True, "max_new_tokens": 150}
# {"repetition_penalty": 1.2, "do_sample": True, "max_new_tokens": 150}
# {"repetition_penalty": 1.3, "do_sample": True, "max_new_tokens": 150}

# {"top_k": 10, "do_sample": True, "max_new_tokens": 150}
# {"top_k": 30, "do_sample": True, "max_new_tokens": 150}
# {"top_k": 50, "do_sample": True, "max_new_tokens": 150}

# {"top_p": 0.7, "do_sample": True, "max_new_tokens": 150}
# {"top_p": 0.8, "do_sample": True, "max_new_tokens": 150}
# {"top_p": 0.9, "do_sample": True, "max_new_tokens": 150}

# {"no_repeat_ngram_size": 2, "do_sample": True, "max_new_tokens": 150}

# {"max_new_tokens": 500}
# {"max_new_tokens": 750}
# {"max_new_tokens": 1000}
# {"max_new_tokens": 1500}
# {"max_new_tokens": 2000}

##### Tree of thought prompt

# MEDIUM MODEL

# {}

# {"temperature": 1e-5}
# {"temperature": 1e-5, "max_new_tokens": 500}
# {"temperature": 1e-5, "max_new_tokens": 750}

# {"top_k": 10, "do_sample": True}
# {"top_k": 30, "do_sample": True}
# {"top_k": 50, "do_sample": True}

# {"top_p": 0.7, "do_sample": True}
# {"top_p": 0.8, "do_sample": True}
# {"top_p": 0.9, "do_sample": True}

# {"repetition_penalty": 1.1, "do_sample": True}
# {"repetition_penalty": 1.2, "do_sample": True}
# {"repetition_penalty": 1.3, "do_sample": True}

# {"no_repeat_ngram_size": 2, "do_sample": True}

# {"temperature": 0.5, "do_sample": True}

# {"temperature": 0.5, "do_sample": True, "max_new_tokens": 150}

# {"repetition_penalty": 1.1, "do_sample": True, "max_new_tokens": 150}
# {"repetition_penalty": 1.2, "do_sample": True, "max_new_tokens": 150}
# {"repetition_penalty": 1.3, "do_sample": True, "max_new_tokens": 150}

# {"top_k": 10, "do_sample": True, "max_new_tokens": 150}
# {"top_k": 30, "do_sample": True, "max_new_tokens": 150}
# {"top_k": 50, "do_sample": True, "max_new_tokens": 150}

# {"top_p": 0.7, "do_sample": True, "max_new_tokens": 150}
# {"top_p": 0.8, "do_sample": True, "max_new_tokens": 150}
# {"top_p": 0.9, "do_sample": True, "max_new_tokens": 150}

# {"no_repeat_ngram_size": 2, "do_sample": True, "max_new_tokens": 150}

# {"max_new_tokens": 500}
# {"max_new_tokens": 750}
# {"max_new_tokens": 1000}
# {"max_new_tokens": 1500}
# {"max_new_tokens": 2000}

##### Persona prompt

# MEDIUM MODEL

# {}

# {"temperature": 1e-5}
# {"temperature": 1e-5, "max_new_tokens": 500}
# {"temperature": 1e-5, "max_new_tokens": 750}

# {"top_k": 10, "do_sample": True}
# {"top_k": 30, "do_sample": True}
# {"top_k": 50, "do_sample": True}

# {"top_p": 0.7, "do_sample": True}
# {"top_p": 0.8, "do_sample": True}
# {"top_p": 0.9, "do_sample": True}

# {"repetition_penalty": 1.1, "do_sample": True}
# {"repetition_penalty": 1.2, "do_sample": True}
# {"repetition_penalty": 1.3, "do_sample": True}

# {"no_repeat_ngram_size": 2, "do_sample": True}

# {"temperature": 0.5, "do_sample": True}

# {"temperature": 0.5, "do_sample": True, "max_new_tokens": 150}

# {"repetition_penalty": 1.1, "do_sample": True, "max_new_tokens": 150}
# {"repetition_penalty": 1.2, "do_sample": True, "max_new_tokens": 150}
# {"repetition_penalty": 1.3, "do_sample": True, "max_new_tokens": 150}

# {"top_k": 10, "do_sample": True, "max_new_tokens": 150}
# {"top_k": 30, "do_sample": True, "max_new_tokens": 150}
# {"top_k": 50, "do_sample": True, "max_new_tokens": 150}

# {"top_p": 0.7, "do_sample": True, "max_new_tokens": 150}
# {"top_p": 0.8, "do_sample": True, "max_new_tokens": 150}
# {"top_p": 0.9, "do_sample": True, "max_new_tokens": 150}

# {"no_repeat_ngram_size": 2, "do_sample": True, "max_new_tokens": 150}

# {"max_new_tokens": 500}
# {"max_new_tokens": 750}
# {"max_new_tokens": 1000}
# {"max_new_tokens": 1500}
# {"max_new_tokens": 2000}


# **************************


##### Zero Shot

# SMALL MODEL

# {}

# {"temperature": 1e-5}

# {"top_k": 10, "do_sample": True}
# {"top_k": 30, "do_sample": True}
# {"top_k": 50, "do_sample": True}

# {"top_p": 0.7, "do_sample": True}
# {"top_p": 0.8, "do_sample": True}
# {"top_p": 0.9, "do_sample": True}

# {"repetition_penalty": 1.1, "do_sample": True}
# {"repetition_penalty": 1.2, "do_sample": True}
# {"repetition_penalty": 1.3, "do_sample": True}

# {"no_repeat_ngram_size": 2, "do_sample": True}

# {"temperature": 0.5, "do_sample": True}

# {"max_new_tokens": 100}
# {"max_new_tokens": 150}
# {"max_new_tokens": 200}

# {"temperature": 1e-5, "max_new_tokens": 100}
# {"temperature": 1e-5, "max_new_tokens": 150}
# {"temperature": 1e-5, "max_new_tokens": 200}

# {"temperature": 0.5, "do_sample": True, "max_new_tokens": 150}

# {"repetition_penalty": 1.1, "do_sample": True, "max_new_tokens": 150}
# {"repetition_penalty": 1.2, "do_sample": True, "max_new_tokens": 150}
# {"repetition_penalty": 1.3, "do_sample": True, "max_new_tokens": 150}

# {"top_k": 10, "do_sample": True, "max_new_tokens": 150}
# {"top_k": 30, "do_sample": True, "max_new_tokens": 150}
# {"top_k": 50, "do_sample": True, "max_new_tokens": 150}

# {"top_p": 0.7, "do_sample": True, "max_new_tokens": 150}
# {"top_p": 0.8, "do_sample": True, "max_new_tokens": 150}
# {"top_p": 0.9, "do_sample": True, "max_new_tokens": 150}

# {"no_repeat_ngram_size": 2, "do_sample": True, "max_new_tokens": 150}

# {"max_new_tokens": 500}
# {"max_new_tokens": 1000}
# {"max_new_tokens": 1500}
# {"max_new_tokens": 2000}

##### Basic Instruction Prompt

# SMALL MODEL

# {}

# {"temperature": 1e-5}

# {"top_k": 10, "do_sample": True}
# {"top_k": 30, "do_sample": True}
# {"top_k": 50, "do_sample": True}

# {"top_p": 0.7, "do_sample": True}
# {"top_p": 0.8, "do_sample": True}
# {"top_p": 0.9, "do_sample": True}

# {"repetition_penalty": 1.1, "do_sample": True}
# {"repetition_penalty": 1.2, "do_sample": True}
# {"repetition_penalty": 1.3, "do_sample": True}

# {"no_repeat_ngram_size": 2, "do_sample": True}

# {"temperature": 0.5, "do_sample": True}

# {"max_new_tokens": 100}
# {"max_new_tokens": 150}
# {"max_new_tokens": 200}

# {"temperature": 1e-5, "max_new_tokens": 100}
# {"temperature": 1e-5, "max_new_tokens": 150}
# {"temperature": 1e-5, "max_new_tokens": 200}

# {"temperature": 0.5, "do_sample": True, "max_new_tokens": 150}

# {"repetition_penalty": 1.1, "do_sample": True, "max_new_tokens": 150}
# {"repetition_penalty": 1.2, "do_sample": True, "max_new_tokens": 150}
# {"repetition_penalty": 1.3, "do_sample": True, "max_new_tokens": 150}

# {"top_k": 10, "do_sample": True, "max_new_tokens": 150}
# {"top_k": 30, "do_sample": True, "max_new_tokens": 150}
# {"top_k": 50, "do_sample": True, "max_new_tokens": 150}

# {"top_p": 0.7, "do_sample": True, "max_new_tokens": 150}
# {"top_p": 0.8, "do_sample": True, "max_new_tokens": 150}
# {"top_p": 0.9, "do_sample": True, "max_new_tokens": 150}

# {"no_repeat_ngram_size": 2, "do_sample": True, "max_new_tokens": 150}

# {"max_new_tokens": 500}
# {"max_new_tokens": 1000}
# {"max_new_tokens": 1500}
# {"max_new_tokens": 2000}

##### Few shot prompt

# SMALL MODEL

# {}

# {"temperature": 1e-5}

# {"max_new_tokens": 100}
# {"max_new_tokens": 150}
# {"max_new_tokens": 200}

# {"temperature": 1e-5, "max_new_tokens": 100}
# {"temperature": 1e-5, "max_new_tokens": 150}
# {"temperature": 1e-5, "max_new_tokens": 200}

# {"temperature": 0.5, "do_sample": True, "max_new_tokens": 150}

# {"top_k": 10, "do_sample": True}
# {"top_k": 30, "do_sample": True}
# {"top_k": 50, "do_sample": True}

# {"top_p": 0.7, "do_sample": True}
# {"top_p": 0.8, "do_sample": True}
# {"top_p": 0.9, "do_sample": True}

# {"repetition_penalty": 1.1, "do_sample": True}
# {"repetition_penalty": 1.2, "do_sample": True}
# {"repetition_penalty": 1.3, "do_sample": True}

# {"no_repeat_ngram_size": 2, "do_sample": True}

# {"temperature": 0.5, "do_sample": True}

# {"repetition_penalty": 1.1, "do_sample": True, "max_new_tokens": 150}
# {"repetition_penalty": 1.2, "do_sample": True, "max_new_tokens": 150}
# {"repetition_penalty": 1.3, "do_sample": True, "max_new_tokens": 150}

# {"top_k": 10, "do_sample": True, "max_new_tokens": 150}
# {"top_k": 30, "do_sample": True, "max_new_tokens": 150}
# {"top_k": 50, "do_sample": True, "max_new_tokens": 150}

# {"top_p": 0.7, "do_sample": True, "max_new_tokens": 150}
# {"top_p": 0.8, "do_sample": True, "max_new_tokens": 150}
# {"top_p": 0.9, "do_sample": True, "max_new_tokens": 150}

# {"no_repeat_ngram_size": 2, "do_sample": True, "max_new_tokens": 150}

# {"max_new_tokens": 500}
# {"max_new_tokens": 1000}
# {"max_new_tokens": 1500}
# {"max_new_tokens": 2000}

##### Chain of thought prompt

# SMALL MODEL

# {}

# {"temperature": 1e-5}

# {"max_new_tokens": 100}
# {"max_new_tokens": 150}
# {"max_new_tokens": 200}

# {"temperature": 1e-5, "max_new_tokens": 100}
# {"temperature": 1e-5, "max_new_tokens": 150}
# {"temperature": 1e-5, "max_new_tokens": 200}

# {"top_k": 10, "do_sample": True}
# {"top_k": 30, "do_sample": True}
# {"top_k": 50, "do_sample": True}

# {"top_p": 0.7, "do_sample": True}
# {"top_p": 0.8, "do_sample": True}
# {"top_p": 0.9, "do_sample": True}

# {"repetition_penalty": 1.1, "do_sample": True}
# {"repetition_penalty": 1.2, "do_sample": True}
# {"repetition_penalty": 1.3, "do_sample": True}

# {"no_repeat_ngram_size": 2, "do_sample": True}

# {"temperature": 0.5, "do_sample": True}

# {"repetition_penalty": 1.1, "do_sample": True, "max_new_tokens": 150}
# {"repetition_penalty": 1.2, "do_sample": True, "max_new_tokens": 150}
# {"repetition_penalty": 1.3, "do_sample": True, "max_new_tokens": 150}

# {"top_k": 10, "do_sample": True, "max_new_tokens": 150}
# {"top_k": 30, "do_sample": True, "max_new_tokens": 150}
# {"top_k": 50, "do_sample": True, "max_new_tokens": 150}

# {"top_p": 0.7, "do_sample": True, "max_new_tokens": 150}
# {"top_p": 0.8, "do_sample": True, "max_new_tokens": 150}
# {"top_p": 0.9, "do_sample": True, "max_new_tokens": 150}

# {"no_repeat_ngram_size": 2, "do_sample": True, "max_new_tokens": 150}

# {"max_new_tokens": 500}
# {"max_new_tokens": 1000}
# {"max_new_tokens": 1500}
# {"max_new_tokens": 2000}

##### Tree of thought prompt

# SMALL MODEL

# {}

# {"temperature": 1e-5}

# {"max_new_tokens": 100}
# {"max_new_tokens": 150}
# {"max_new_tokens": 200}

# {"temperature": 1e-5, "max_new_tokens": 100}
# {"temperature": 1e-5, "max_new_tokens": 150}
# {"temperature": 1e-5, "max_new_tokens": 200}

# {"top_k": 10, "do_sample": True}
# {"top_k": 30, "do_sample": True}
# {"top_k": 50, "do_sample": True}

# {"top_p": 0.7, "do_sample": True}
# {"top_p": 0.8, "do_sample": True}
# {"top_p": 0.9, "do_sample": True}

# {"repetition_penalty": 1.1, "do_sample": True}
# {"repetition_penalty": 1.2, "do_sample": True}
# {"repetition_penalty": 1.3, "do_sample": True}

# {"no_repeat_ngram_size": 2, "do_sample": True}

# {"temperature": 0.5, "do_sample": True}

# {"repetition_penalty": 1.1, "do_sample": True, "max_new_tokens": 150}
# {"repetition_penalty": 1.2, "do_sample": True, "max_new_tokens": 150}
# {"repetition_penalty": 1.3, "do_sample": True, "max_new_tokens": 150}

# {"top_k": 10, "do_sample": True, "max_new_tokens": 150}
# {"top_k": 30, "do_sample": True, "max_new_tokens": 150}
# {"top_k": 50, "do_sample": True, "max_new_tokens": 150}

# {"top_p": 0.7, "do_sample": True, "max_new_tokens": 150}
# {"top_p": 0.8, "do_sample": True, "max_new_tokens": 150}
# {"top_p": 0.9, "do_sample": True, "max_new_tokens": 150}

# {"no_repeat_ngram_size": 2, "do_sample": True, "max_new_tokens": 150}

# {"max_new_tokens": 500}
# {"max_new_tokens": 1000}
# {"max_new_tokens": 1500}
# {"max_new_tokens": 2000}

##### Persona prompt

# SMALL MODEL

# {}

# {"temperature": 1e-5}

# {"max_new_tokens": 100}
# {"max_new_tokens": 150}
# {"max_new_tokens": 200}

# {"temperature": 1e-5, "max_new_tokens": 100}
# {"temperature": 1e-5, "max_new_tokens": 150}
# {"temperature": 1e-5, "max_new_tokens": 200}

# {"top_k": 10, "do_sample": True}
# {"top_k": 30, "do_sample": True}
# {"top_k": 50, "do_sample": True}

# {"top_p": 0.7, "do_sample": True}
# {"top_p": 0.8, "do_sample": True}
# {"top_p": 0.9, "do_sample": True}

# {"repetition_penalty": 1.1, "do_sample": True}
# {"repetition_penalty": 1.2, "do_sample": True}
# {"repetition_penalty": 1.3, "do_sample": True}

# {"no_repeat_ngram_size": 2, "do_sample": True}

# {"temperature": 0.5, "do_sample": True}

# {"repetition_penalty": 1.1, "do_sample": True, "max_new_tokens": 150}
# {"repetition_penalty": 1.2, "do_sample": True, "max_new_tokens": 150}
# {"repetition_penalty": 1.3, "do_sample": True, "max_new_tokens": 150}

# {"top_k": 10, "do_sample": True, "max_new_tokens": 150}
# {"top_k": 30, "do_sample": True, "max_new_tokens": 150}
# {"top_k": 50, "do_sample": True, "max_new_tokens": 150}

# {"top_p": 0.7, "do_sample": True, "max_new_tokens": 150}
# {"top_p": 0.8, "do_sample": True, "max_new_tokens": 150}
# {"top_p": 0.9, "do_sample": True, "max_new_tokens": 150}

# {"no_repeat_ngram_size": 2, "do_sample": True, "max_new_tokens": 150}

# {"max_new_tokens": 500}
# {"max_new_tokens": 1000}
# {"max_new_tokens": 1500}
# {"max_new_tokens": 2000}


# Hyperparameters to be passed to model (note commented out ones above show previous runs)
LLM_HYPERPARAMETERS = {"temperature": 1e-5, "max_new_tokens": 500}

In [ ]:
device_id = 0 if torch.cuda.is_available() else -1

# Initialise the llm using the model name in use, passing the hyperparameters and device_id (to ensure use of GPU if available)
llm = HuggingFacePipeline.from_model_id(
    model_id=LLM_IN_USE_NAME,
    task="text-generation",
    device=device_id,
    pipeline_kwargs=LLM_HYPERPARAMETERS
)

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

...

model-00003-of-00003.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/27.3M [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
# The different prompt types used for the RAG model runs

##### Zero Shot

zero_shot_rag_prompt = """Use the following pieces of context to answer the question.

{context}

Question: {question}

Helpful Answer:
"""

##### Persona

persona_rag_prompt = """You are a warm, empathetic, and knowledgeable assistant for the Irish Citizen's Information service. Respond clearly, simply, and accurately, and use the following pieces of context to answer the question as well as your existing knowledge of official information relevant to Ireland.

{context}

Question: {question}

Helpful Answer:
"""


##### Few shot

few_shot_rag_prompt = """Answer the question carefully, using the following pieces of context and examples below to guide the style, structure and level of detail expected in the answer. ONLY provide the answer to Example 3. Follow the same structure and level of detail as Example 1 and Example 2 - short, factual, and readable. Do not list more than necessary. Stop after answering Example 3.

{context}

Example 1:

Q: What documents do I need to apply online for a driver's license in Ireland?
A: You need a Public Services Card and verified MyGovID, proof of your address if your address is different to the one you provided when you got your Public Services Card, proof that you are normally resident in Ireland if you are not an EU or EEA citizen, a medical report form, dated within one month if required, and a Certificate of Professional Competence (CPC) for professional drivers.

Example 2:

Q: What criteria do I need to meet to qualify for unemployment benefits in Ireland?
A: You must be under 66 years of age, or under 70 years and deferring your State Pension (Contributory), be unemployed (you must be unemployed for at least 4 days out of 7), have had a substantial loss of employment and as a result be unemployed for at least 4 days out of 7 (this does not apply to casual workers and part-time fire fighters), be capable of work, be available for and genuinely seeking work, and have enough social insurance (PRSI) contributions.

Example 3:
Q: {question}
A:
"""


##### Basic Instruction Prompt

instruction_rag_prompt = """Use the following pieces of context to answer the question at the end. Follow these rules carefully:
1. If you don't know the answer, don't try to make up an answer. Just say "I can't find the final answer but you may want to check the following links".
2. If you find the answer, write the answer in a concise way with five sentences maximum.
3. DO NOT copy entire sections, lists, or extracted content. Instead, extract only the relevant poionts to answer the question effectively.
4. Exclude any headings, subheadings, or introductory information. Only include the final, relevant points in your response.
5. DO NOT include extracted documents section.

{context}

Question: {question}

Helpful Answer:
"""

##### Chain of thought Prompt

chain_of_thought_rag_prompt = """Use the following pieces of context to reason step by step before answering the question.

{context}

Question: {question}

Begin by enclosing all thoughts within <thinking> tags, exploring multiple angles and approaches. Break down the solution into clear steps within <step> tags. Start with a 10-step budget, requesting more for complex problems if needed. Use <count> tags after each step to show the remaining budget. Stop when reaching 0. Continuously adjust your reasoning based on intermediate results and reflections, adapting your strategy as you progress. Regularly evaluate progress using <reflection> tags. Be critical and honest about your reasoning process. Assign a quality score between 0.0 and 1.0 using <reward> tags after each reflection. Use this to guide your approach: 0.8+: Continue current approach 0.5-0.7: Consider minor adjustments Below 0.5: Seriously consider backtracking and trying a different approach If unsure or if reward score is low, backtrack and try a different approach, explaining your decision within <thinking> tags. Explore multiple solutions individually if possible, comparing approaches in reflections. Use thoughts as a scratchpad, writing out all calculations and reasoning explicitly. Synthesize the final answer within <answer> tags, providing a clear, concise summary. Conclude with a final reflection on the overall solution, discussing effectiveness, challenges, and solutions. Assign a final reward score.

Helpful Answer:
"""

##### Tree of thought Prompt

tree_of_thought_rag_prompt = """Use the following pieces of context to reason step by step before answering the question.

{context}

Question: {question}

Begin by enclosing all thoughts within <thinking> tags. Instead of a single linear chain, explicitly generate and explore multiple branches of reasoning as separate <branch id="X"> elements, each representing a distinct approach or angle.

Within each <branch>:
- Break down the reasoning into clear <step> tags.
- After each step, include a <count> tag showing the remaining step budget (start with 10 steps per branch; request more if needed).
- Use <reflection> tags periodically within branches to critically evaluate progress.
- Assign a <reward> score (0.0 to 1.0) after reflections to assess quality of that branch's approach:
   - 0.8+ Continue this branch
   - 0.5-0.7 Consider minor adjustments
   - Below 0.5 Backtrack or abandon this branch
- If reward is low or you're unsure, backtrack and try a different <branch>, explaining the reasoning inside new <thinking> tags.
- Use thoughts as a scratchpad: show calculations, considerations, pros/cons explicitly.

Once multiple branches have been explored and evaluated, compare and synthesize insights in a <comparison> tag, weighing strengths and weaknesses.
Finally, provide a consolidated final answer inside <answer> tags - a clear, consise summary based on the best branch or synthesis.
Conclude with an overall <reflection> and a final <reward> score reflecting the whole process's effectiveness.
"""

prompt_dict_rag = {
      ZERO_SHOT_PROMPT_TYPE: zero_shot_rag_prompt,
      PERSONA_PROMPT_TYPE: persona_rag_prompt,
      FEW_SHOT_PROMPT_TYPE: few_shot_rag_prompt,
      INSTRUCTION_PROMPT_TYPE: instruction_rag_prompt,
      CHAIN_OF_THOUGHT_PROMPT_TYPE: chain_of_thought_rag_prompt,
      TREE_OF_THOUGHT_PROMPT_TYPE: tree_of_thought_rag_prompt
  }


prompt_template_rag = get_prompt_by_type(PROMPT_TYPE_IN_USE, prompt_dict_rag)

PROMPT_RAG = PromptTemplate(
 template=prompt_template_rag, input_variables=["context", "question"]
)

In [ ]:
# The different prompt types used for the LLM model runs

##### Zero Shot

zero_shot_llm_prompt = """Use your existing knowledge to answer the question.

Question: {question}

Helpful Answer:
"""

##### Persona

persona_llm_prompt = """You are a warm, empathetic, and knowledgeable assistant for the Irish Citizen's Information service. Respond clearly, simply, and accurately, and use your existing knowledge of official information relevant to Ireland.

Question: {question}

Helpful Answer:
"""


##### Few shot

few_shot_llm_prompt = """Answer the question carefully, using the examples below to guide the style, structure and level of detail expected in the answer. ONLY provide the answer to Example 3. Follow the same structure and level of detail as Example 1 and Example 2 - short, factual, and readable. Do not list more than necessary. Stop after answering Example 3.

Example 1:

Q: What documents do I need to apply online for a driver's license in Ireland?
A: You need a Public Services Card and verified MyGovID, proof of your address if your address is different to the one you provided when you got your Public Services Card, proof that you are normally resident in Ireland if you are not an EU or EEA citizen, a medical report form, dated within one month if required, and a Certificate of Professional Competence (CPC) for professional drivers.

Example 2:

Q: What criteria do I need to meet to qualify for unemployment benefits in Ireland?
A: You must be under 66 years of age, or under 70 years and deferring your State Pension (Contributory), be unemployed (you must be unemployed for at least 4 days out of 7), have had a substantial loss of employment and as a result be unemployed for at least 4 days out of 7 (this does not apply to casual workers and part-time fire fighters), be capable of work, be available for and genuinely seeking work, and have enough social insurance (PRSI) contributions.

Example 3:
Q: {question}
A:
"""

##### Basic Instruction Prompt

instruction_llm_prompt = """Answer the following question carefully. Follow these rules carefully:
1. If you don't know the answer, don't try to make up an answer. Just say "I can't find the final answer but you may want to check the following links".
2. If you find the answer, write the answer in a concise way with five sentences maximum.
3. DO NOT copy entire sections, lists, or extracted content. Instead, extract only the relevant poionts to answer the question effectively.
4. Exclude any headings, subheadings, or introductory information. Only include the final, relevant points in your response.

Question: {question}

Helpful Answer:
"""

##### Chain of thought Prompt

chain_of_thought_llm_prompt = """Reason step by step before answering the question.

Question: {question}

Begin by enclosing all thoughts within <thinking> tags, exploring multiple angles and approaches. Break down the solution into clear steps within <step> tags. Start with a 10-step budget, requesting more for complex problems if needed. Use <count> tags after each step to show the remaining budget. Stop when reaching 0. Continuously adjust your reasoning based on intermediate results and reflections, adapting your strategy as you progress. Regularly evaluate progress using <reflection> tags. Be critical and honest about your reasoning process. Assign a quality score between 0.0 and 1.0 using <reward> tags after each reflection. Use this to guide your approach: 0.8+: Continue current approach 0.5-0.7: Consider minor adjustments Below 0.5: Seriously consider backtracking and trying a different approach If unsure or if reward score is low, backtrack and try a different approach, explaining your decision within <thinking> tags. Explore multiple solutions individually if possible, comparing approaches in reflections. Use thoughts as a scratchpad, writing out all calculations and reasoning explicitly. Synthesize the final answer within <answer> tags, providing a clear, concise summary. Conclude with a final reflection on the overall solution, discussing effectiveness, challenges, and solutions. Assign a final reward score.

Helpful Answer:
"""

##### Tree of thought Prompt

tree_of_thought_llm_prompt = """Reason step by step before answering the question.

Question: {question}

Begin by enclosing all thoughts within <thinking> tags. Instead of a single linear chain, explicitly generate and explore multiple branches of reasoning as separate <branch id="X"> elements, each representing a distinct approach or angle.

Within each <branch>:
 - Break down the reasoning into clear <step> tags.
 - After each step, include a <count> tag showing the remaining step budget (start with 10 steps per branch; request more if needed).
 - Use <reflection> tags periodically within branches to critically evaluate progress.
 - Assign a <reward> score (0.0 to 1.0) after reflections to assess quality of that branch's approach:
   - 0.8+ Continue this branch
   - 0.5-0.7 Consider minor adjustments
   - Below 0.5 Backtrack or abandon this branch
 - If reward is low or you're unsure, backtrack and try a different <branch>, explaining the reasoning inside new <thinking> tags.
 - Use thoughts as a scratchpad: show calculations, considerations, pros/cons explicitly.

Once multiple branches have been explored and evaluated, compare and synthesize insights in a <comparison> tag, weighing strengths and weaknesses.
Finally, provide a consolidated final answer inside <answer> tags - a clear, consise summary based on the best branch or synthesis.
Conclude with an overall <reflection> and a final <reward> score reflecting the whole process's effectiveness.
"""

prompt_dict_llm = {
      ZERO_SHOT_PROMPT_TYPE: zero_shot_llm_prompt,
      PERSONA_PROMPT_TYPE: persona_llm_prompt,
      FEW_SHOT_PROMPT_TYPE: few_shot_llm_prompt,
      INSTRUCTION_PROMPT_TYPE: instruction_llm_prompt,
      CHAIN_OF_THOUGHT_PROMPT_TYPE: chain_of_thought_llm_prompt,
      TREE_OF_THOUGHT_PROMPT_TYPE: tree_of_thought_llm_prompt
  }

prompt_template_llm = get_prompt_by_type(PROMPT_TYPE_IN_USE, prompt_dict_llm)

PROMPT_LLM = PromptTemplate(
 template=prompt_template_llm, input_variables=["question"]
)

In [ ]:
# If re-ranker set, initialise re-ranking model, otherwise set to None
if RE_RANKER:
  RERANKER = RAGPretrainedModel.from_pretrained("colbert-ir/colbertv2.0")
else:
  RERANKER = None

artifact.metadata: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/405 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/colbert/utils/amp.py:12: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler()


In [ ]:
def post_process_helpful_answer_prompt(answer: str) -> str:
  """
  Extract actual answer text from generated LLM output

  Args:
    answer (str): LLM output
  
  Returns:
    str: actual answer text extracted from generated LLM output
  """

  # Use flag to check start of answer, if present, extract from last index position to end of document, otherwise just return full generated answer
  if "Helpful Answer:" in answer:
    final_response = answer.split("Helpful Answer:")[-1].strip()
  else:
    final_response = answer.strip()

  return final_response

In [ ]:
def post_process_helpful_answer_prompt_chain_of_thought(answer: str) -> str:
  """
  Extract actual answer text from chain of thought & tree of thought prompts

  Args:
    answer (str): LLM output

  Returns:
    str: actual answer text extracted from generated LLM output

  """

  # Use re package to find matches for <answer> </answer> tags (used in prompt)
  start_matches = list(re.finditer(r"<answer>", answer, flags=re.IGNORECASE))

  end_matches = list(re.finditer(r"</answer>", answer, flags=re.IGNORECASE))


  # If tags not found, return empty string
  if len(start_matches) < 2:
    return ""


  # Otherwise, extract answer from between these tags and return
  start_index = start_matches[-1].end()

  if end_matches:
    end_after_start = [m for m in end_matches if m.start() > start_index]

    if end_after_start:
      end_index = end_after_start[-1].start()
    else:
      end_index = len(answer)
  else:
    end_index = len(answer)

  return answer[start_index:end_index].strip()

In [ ]:
def post_process_few_shot(answer: str) -> str:
  """
  Extract actual answer text from few-shot prompts

  Args:
    answer (str): LLM output

  Returns:
    str: actual answer text extracted from generated LLM output

  """

  # Use marker "A:" used in prompt to outline start of generated answer
  marker = "A:"

  # If this marker not found in text, return empty string
  if marker not in answer:
    return ""

  # Return from last index of marker to end of text
  return answer.split(marker)[-1].strip()

In [ ]:
def trim_generated_output_to_last_complete_sentence(generated_text: str) -> str:
  """
  Trim generated output to last complete sentence to ensure that generated output is not partial sentence
  """

  # Use regex match to determine if output is character other than . or ! or ?
  regex_match = re.search(r"([.!?])[^.!?]*$", generated_text)

  # If there is a match, output mustn't have ended successfully so trim output to last index + 1
  if regex_match:
    return generated_text[: regex_match.start(1) + 1]

  return generated_text

In [ ]:
def track_runtime(func: Callable[..., Any]):
  """
  Decorator to track the runtime of a function and add metadata around timing to the output

  Args:
    func (Callable[..., Any]): the function to be wrapped
  
  Returns:
    Callable[..., Any]: A wrapped function that returns a tuple with output and runtime information
  """
  @wraps(func)
  def wrapper(*args, **kwargs):
    # Capture start time of function
    start_time = time.time()

    # Call function
    try:
      result = func(*args, **kwargs)
    except Exception as e:
      print("Error in function execution: {}".format(e))
      raise e

    # Capture end time of function
    end_time = time.time()

    # Extract run time by subtracting start time from end time
    run_time = end_time - start_time

    # Return both function outputs of wrapped function and add run time information to returned outputs
    if isinstance(result, tuple):
      raw_output = result[0]

      formatted_output = result[1] if len(result) > 1 else []

      relevant_docs = result[2] if len(result) > 2 else []

      timing_info = result[3] if len(result) > 3 else {}

      post_process_function = result[4] if len(result) > 4 else {}
    else:
      raw_output = result
      formatted_output = result
      relevant_docs = []
      timing_info = {}
      post_process_function = {}

    return (raw_output, formatted_output, relevant_docs, timing_info, post_process_function, run_time)

  return wrapper

In [ ]:
def extract_relevant_document_chunks(question: str, knowledge_index: FAISS, num_retrieved_docs: int, num_docs_final: int, reranker: Any) -> List[Document]:
  """
  Extract relevant documents from provided vector store for given question value

  Args:
    question (str): actual question against which relevant documents should be retrieved
    knowledge_index (FAISS): vector db variable
    num_retrieved_docs (int): number of docs to retrieve
    num_docs_final (int): number of final docs to return (if re-ranking enabled)
    reranker (Any): re-ranker model if provided
  
  Returns:
    List[Document]: 
  """

  # Extract relevant documents using the similarity_search function
  relevant_docs = knowledge_index.similarity_search(query=question, k=num_retrieved_docs)

  # Extract content from retrieved documents
  relevant_docs = [doc.page_content for doc in relevant_docs]

  # If re-ranking enabled, use reranker model to rerank the retrieved documents
  if reranker:
    print("=> Reranking documents...")
    relevant_docs = reranker.rerank(question, relevant_docs, k=num_docs_final)
    relevant_docs = [doc["content"] for doc in relevant_docs]

  # Filter to number of final documents value
  relevant_docs = relevant_docs[:num_docs_final]

  # Return retrieved documents
  return relevant_docs

In [ ]:
# This cell was used to test that the re-ranking was working as expected
# Use get_file_paths_matching_regex function to match files within content directory with golden results and rag within the file name
rag_results_file_paths = get_file_paths_matching_regex("/content", r"(?=.*golden_results)(?=.*rag)")


for results_file_path in rag_results_file_paths:
  current_result_info = read_json_file(results_file_path)

  for i, info in enumerate(current_result_info["results"]):
    retrieved_documents = extract_relevant_document_chunks(info["question"], vectorstore_medium_model, 20, 5, RERANKER)
    info["final_documents"] = retrieved_documents

  write_json_file(results_file_path, current_result_info)

Successfully loaded content from /content/golden_results_few_shot_mistralai_Mistral-7B-Instruct-v0.3_NVIDIA_A100-SXM4-40GB_rag_temp0_topk0_topP0_repPenalty0_noRepeatNGrams0_max750_24-06-2025_14-08-06.json file
=> Reranking documents...


100%|██████████| 1/1 [00:00<00:00, 36.71it/s]


=> Reranking documents...


100%|██████████| 1/1 [00:00<00:00, 35.84it/s]


=> Reranking documents...


...

=> Reranking documents...


100%|██████████| 1/1 [00:00<00:00, 41.12it/s]


=> Reranking documents...


100%|██████████| 1/1 [00:00<00:00, 41.37it/s]

Successfully saved content to /content/golden_results_few_shot_mistralai_Mistral-7B-Instruct-v0.3_NVIDIA_A100-SXM4-40GB_rag_temp0_topk0_topP0_repPenalty0_noRepeatNGrams0_max2000_24-06-2025_13-51-24.json


In [ ]:
# Note use of decorator function here to ensure run time information extracted
@track_runtime
def answer_with_rag(
    question: str,
    llm,
    knowledge_index: FAISS,
    prompt_template,
    reranker: Optional[RAGPretrainedModel] = None,
    num_retrieved_docs: int = 30,
    num_docs_final: int = 5,
    post_process_function = None,
):
    """
    Function to enable RAG-enabled LLM inference

    Args:
      question (str): the user query
      llm: the model used for inference
      knowledge_index (FAISS): vector db store generated with citizen information documents
      prompt_template: prompt template used to be passed to the model
      reranker (Optional[RAGPretrainedModel]): re-ranking model
      num_retrieved_docs (int): number of initial relevant docs to be retrieved
      num_docs_final (int): number of final documents to be returned
      post_process_function: post process function to be called on generated output
    
    Returns:
      answer (str): raw generated output
      formatted_answer (str): output after post-processing completed
      relevant_docs: retrieved documents returned
      timing_info: timing information which stores run-time etc.
      post_process_function: the function used to post-process generated output 
    """

    # Initislise empty dict for timing information
    timing_info = {}

    # Extract documents with the retriever
    print("=> Retrieving documents...")

    # Keep track of retrieval start time
    start_retrieval = time.time()

    # Get relevant docs
    relevant_docs = knowledge_index.similarity_search(query=question, k=num_retrieved_docs)

    # Keep track of retrieval end time
    end_retrieval = time.time()

    # Set timing info retrieval duration time
    timing_info["retrieval_duration"] = end_retrieval - start_retrieval


    # Extract content for retrieved documents
    relevant_docs = [doc.page_content for doc in relevant_docs]

    # Re-rank results if enabled
    if reranker:
        print("=> Reranking documents...")
        # Keep track of re-ranking start time
        start_rerank = time.time()

        # Actually re-rank docs
        relevant_docs = reranker.rerank(question, relevant_docs, k=num_docs_final)

        # Keep track of re-ranking start time
        end_rerank = time.time()

        # Set timing info re-rank duration time
        timing_info["rerank_duration"] = end_rerank - start_rerank

        # Extract content for re-ranked documents
        relevant_docs = [doc["content"] for doc in relevant_docs]

    # Filter docs to index position of num_docs_final
    relevant_docs = relevant_docs[:num_docs_final]

    # Construct the final prompt with the extracted documents text included
    context = "\nExtracted documents:\n"
    context += "".join([f"Document {str(i)}:::\n" + doc for i, doc in enumerate(relevant_docs)])

    final_prompt = prompt_template.format(question=question, context=context)

    # Call model to generate answer
    print("=> Generating answer...")

    # Keep track of start time when actual model called 
    start_gen = time.time()
    answer = llm(final_prompt)

    # Keep track of end time when actual model returns output
    end_gen = time.time()

    # Set generation timing info on timing info dict
    timing_info["generation_duration"] = end_gen - start_gen

    # Extract number output tokens and time per output tokens if available (NOTE: this wasn't used in final analysis as latency alone was sufficient to show differences)
    if hasattr(llm.pipeline.tokenizer, "encode"):
      output_tokens = llm.pipeline.tokenizer.encode(answer, add_special_tokens=False)
      timing_info["number_output_tokens"] = len(output_tokens)
      timing_info["time_per_output_token"] = timing_info["generation_duration"] / timing_info["number_output_tokens"] if output_tokens else None
    else:
      timing_info["number_output_tokens"] = -1
      timing_info["time_per_output_token"] = None


    # Format generated output by calling post process function
    formatted_answer = post_process_function(answer) if post_process_function else answer

    # Then once again post-process by replacing any numbered points with - bullets
    formatted_answer = re.sub(r'\b([a-zA-Z0-9])\.\s+', "- ", formatted_answer).strip()

    # Trim output to last complete sentence
    formatted_answer = trim_generated_output_to_last_complete_sentence(formatted_answer)

    # Return raw answer, formatted answer, retrieved documents, timing information and post process function used for post-processing
    return answer, formatted_answer, relevant_docs, timing_info, post_process_function

In [ ]:
# Note use of decorator function here to ensure run time information extracted
@track_runtime
def answer_without_rag(
    question: str,
    llm,
    prompt_template,
    post_process_function = None,
    ):
    """
    Function for LLM inference

    Args:
      question (str): actual user query
      llm: model to be used for inference
      prompt_template: prompt to be provided to llm
      post_process_function: function to be used for post-processing of output
    
    Returns:
      answer: raw output from LLM model
      formatted_answer: output after post-processing
      retrieved documents: empty list as not used in non-RAG based inference
      timing_info: timing information which stores inference run time
      post_process_function: function used for post-processing
    """

    # Initialise empty timing info dict
    timing_info = {}

    # Get prompt type
    final_prompt = prompt_template.format(question=question)

    # Inference
    print("=> Generating answer...")

    # Keep track of inference start time
    start_gen = time.time()

    # Actually call llm with prompt
    answer = llm(final_prompt)

    # Keep track of inference end time
    end_gen = time.time()

    # Set inferencing duration on timing info dict
    timing_info["generation_duration"] = end_gen - start_gen

    # If available, extract number of output tokens and time per output token (NOTE: in final output, didn't need these as latency alone showed difference)
    if hasattr(llm.pipeline.tokenizer, "encode"):
      output_tokens = llm.pipeline.tokenizer.encode(answer, add_special_tokens=False)
      timing_info["number_output_tokens"] = len(output_tokens)
      timing_info["time_per_output_token"] = timing_info["generation_duration"] / timing_info["number_output_tokens"] if output_tokens else None
    else:
      timing_info["number_output_tokens"] = -1
      timing_info["time_per_output_token"] = None


    # Call post-processing function to format answer
    formatted_answer = post_process_function(answer) if post_process_function else answer


    # Then once again post-process by replacing any numbered points with - bullets
    formatted_answer = re.sub(r'\b([a-zA-Z0-9])\.\s+', "- ", formatted_answer).strip()

    # Trim output to last complete sentence
    formatted_answer = trim_generated_output_to_last_complete_sentence(formatted_answer)

    # Return raw answer, formatted answer, empty list as no retrieval here, timing information and post process function used for post-processing
    return answer, formatted_answer, [], timing_info, post_process_function

In [ ]:
# Initialise dict with prompt types as keys and prompt templates stored as values
post_process_function_dict = {
    ZERO_SHOT_PROMPT_TYPE: post_process_helpful_answer_prompt,
    INSTRUCTION_PROMPT_TYPE: post_process_helpful_answer_prompt,
    PERSONA_PROMPT_TYPE: post_process_helpful_answer_prompt,
    TREE_OF_THOUGHT_PROMPT_TYPE: post_process_helpful_answer_prompt_chain_of_thought,
    CHAIN_OF_THOUGHT_PROMPT_TYPE: post_process_helpful_answer_prompt_chain_of_thought,
    FEW_SHOT_PROMPT_TYPE: post_process_few_shot,
}

In [ ]:
# Extract post processing function to be used based on current prompt type
post_process_function = get_post_process_function_by_prompt_type(PROMPT_TYPE_IN_USE, post_process_function_dict)

In [ ]:
# Test call for RAG workflow to ensure everything working as expected
raw_answer_rag, formatted_answer_rag, relevant_docs, timing_info_rag, post_process_function_rag, total_run_time_rag = retry_hugging_face_inference(answer_with_rag, "Who is entitled to Irish citizenship?", llm, vectorstore_medium_model, PROMPT_RAG, RERANKER, NUMBER_RETRIEVED_DOCS, NUMBER_FINAL_DOCS, post_process_function)

=> Retrieving documents...
=> Generating answer...


/tmp/ipython-input-43-2609863738.py:44: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  answer = llm(final_prompt)
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:629: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `1e-05` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


In [ ]:
# Print post-processed output to validate
print(formatted_answer_rag)

* A person born in Ireland or a child born abroad to an Irish parent or parents
* A person who has been granted Irish citizenship by naturalisation
* A person who has been granted Irish citizenship by registration
* A person who has been granted Irish citizenship by descent
* A person who has been granted Irish citizenship by marriage to an Irish citizen
* A person who has been granted Irish citizenship by operation of law
* A person who has been granted Irish citizenship by order of the President


Answer:

* A person born in Ireland or a child born abroad to an Irish parent or parents
* A person who has been granted Irish citizenship by naturalisation
* A person who has been granted Irish citizenship by registration
* A person who has been granted Irish citizenship by descent
* A person who has been granted Irish citizenship by marriage to an Irish citizen
* A person who has been granted Irish citizenship by operation of law
* A person who has been granted Irish citizenship by order 

In [ ]:
# Print raw output to visualise full response from model
print(raw_answer_rag)

Answer the question carefully, using the following pieces of context and examples below to guide the style, structure and level of detail expected in the answer. ONLY provide the answer to Example 3. Follow the same structure and level of detail as Example 1 and Example 2 - short, factual, and readable. Do not list more than necessary. Stop after answering Example 3.


Extracted documents:


Example 1:

Q: What documents do I need to apply online for a driver's license in Ireland?
A: You need a Public Services Card and verified MyGovID, proof of your address if your address is different to the one you provided when you got your Public Services Card, proof that you are normally resident in Ireland if you are not an EU or EEA citizen, a medical report form, dated within one month if required, and a Certificate of Professional Competence (CPC) for professional drivers.

Example 2:

Q: What criteria do I need to meet to qualify for unemployment benefits in Ireland?
A: You must be under 66 

In [ ]:

# Test call for LLM workflow to ensure everything working as expected
raw_answer_llm, formatted_answer_llm, _, timing_info_llm, post_process_function_llm, total_run_time_llm = retry_hugging_face_inference(answer_without_rag, "Who is entitled to Irish citizenship?", llm, PROMPT_LLM, post_process_function)

=> Generating answer...


In [ ]:
# Print post-processed output to validate
print(formatted_answer_llm)

In [ ]:
# Print raw output to visualise full response from model
print(raw_answer_llm)

Reason step by step before answering the question.

Question: Who is entitled to Irish citizenship?

Begin by enclosing all thoughts within <thinking> tags. Instead of a single linear chain, explicitly generate and explore multiple branches of reasoning as separate <branch id="X"> elements, each representing a distinct approach or angle.

Within each <branch>:
 - Break down the reasoning into clear <step> tags.
 - After each step, include a <count> tag showing the remaining step budget (start with 10 steps per branch; request more if needed).
 - Use <reflection> tags periodically within branches to critically evaluate progress.
 - Assign a <reward> score (0.0 to 1.0) after reflections to assess quality of that branch's approach:
   - 0.8+ Continue this branch
   - 0.5-0.7 Consider minor adjustments
   - Below 0.5 Backtrack or abandon this branch
 - If reward is low or you're unsure, backtrack and try a different <branch>, explaining the reasoning inside new <thinking> tags.
 - Use thou

In [ ]:
# Before completing full run, sanity check that correct prompts and post-process function are in use
print(PROMPT_RAG)

print("*" * 50)

print(PROMPT_LLM)

print ("*" * 50)

print(post_process_function)

input_variables=['context', 'question'] input_types={} partial_variables={} template="Answer the question carefully, using the following pieces of context and examples below to guide the style, structure and level of detail expected in the answer. ONLY provide the answer to Example 3. Follow the same structure and level of detail as Example 1 and Example 2 - short, factual, and readable. Do not list more than necessary. Stop after answering Example 3.\n\n{context}\n\nExample 1:\n\nQ: What documents do I need to apply online for a driver's license in Ireland?\nA: You need a Public Services Card and verified MyGovID, proof of your address if your address is different to the one you provided when you got your Public Services Card, proof that you are normally resident in Ireland if you are not an EU or EEA citizen, a medical report form, dated within one month if required, and a Certificate of Professional Competence (CPC) for professional drivers.\n\nExample 2:\n\nQ: What criteria do I ne

## Golden Dataset - Generate answers with LLM + RAG

In [ ]:
# Open the golden_dataset.json file and read contents
with open("golden_dataset.json", "r", encoding="utf-8") as f:
      golden_dataset = json.load(f)


# If set to True, run full golden dataset inference for all six prompt types
if RUN_ALL_PROMPTS:
  print("Running all prompts...")
  print("*" * 100)
  print("\n")
  print(LLM_HYPERPARAMETERS)
  # Loop over prompts
  for prompt_type, prompt_text in prompt_dict_rag.items():
    print("Running {} prompt type".format(prompt_type))
    print("*" * 100)
    print("\n")
    rag_answers_large_model_golden_questions = []


    # Get prompt template contents for current prompt type
    prompt_template_rag = get_prompt_by_type(prompt_type, prompt_dict_rag)

    # Construct prompt template
    PROMPT_RAG = PromptTemplate(
      template=prompt_template_rag, input_variables=["context", "question"]
    )
    
    # Extract post-process function for current prompt type
    post_process_function = get_post_process_function_by_prompt_type(prompt_type, post_process_function_dict)

    # Loop over golden dataset, generate output for each question within and store generated outputs
    for i in golden_dataset:
      raw_rag_answer, formatted_rag_answer, relevant_docs, timing_info, post_process_function_rag, run_time_rag = retry_hugging_face_inference(answer_with_rag, i["question"], llm, vectorstore_medium_model, PROMPT_RAG, RERANKER, NUMBER_RETRIEVED_DOCS, NUMBER_FINAL_DOCS, post_process_function)
      post_process_function_rag_string = inspect.getsource(post_process_function_rag)
      rag_answers_large_model_golden_questions.append({"ground_truth": i["answer"], "question": i["question"], "raw_answer": raw_rag_answer, "answer": formatted_rag_answer, "total_run_time": run_time_rag, "timing_info": timing_info, "post_process_function": post_process_function_rag_string, "final_documents": relevant_docs})

    instance_name = get_instance_name()

    # Extract unique json file path name
    golden_large_model_file_path = generate_json_file_name(prompt_type, LLM_IN_USE_NAME, "rag", TEMPERATURE, TOP_K, TOP_P, REPETITION_PENALTY, NO_REPEAT_N_GRAMS, MAX_TOKENS, instance_name, "golden")


    # initialise dict with all info to be stored in results file for current run
    rag_experiment_results = {
    "hyperparameters": LLM_HYPERPARAMETERS,
    "model": LLM_IN_USE_NAME,
    "results": rag_answers_large_model_golden_questions,
    "colab_instance": instance_name,
    "prompt_template": prompt_template_rag,
    "prompt_type": prompt_type,
    "knowledge_index": "FAISS",
    "reranker": RE_RANKER,
    "num_retrieved_docs": NUMBER_RETRIEVED_DOCS,
    "num_docs_final":  NUMBER_FINAL_DOCS,
    "embeddings": { "model_name": MEDIUM_TOKENISER_MODEL_NAME,
                    "model_kwargs": {"device": device,},
                    "encode_kwargs": {"normalize_embeddings": True, "multi_process": False}
                   },
    "chunk_size": CHUNK_SIZE,
    "chunk_overlap": int(CHUNK_SIZE / 10),
    "add_start_index": True,
    "strip_whitespace": True,
    "separators": MARKDOWN_SEPARATORS,
    }

    # write json file with this information
    write_json_file(golden_large_model_file_path, rag_experiment_results)

# If set to False, only run for current prompt template
else:
  rag_answers_large_model_golden_questions = []

  # Loop over each question in the dataset, generate outputs and append information to list for each qa pair
  for i in golden_dataset:
    print("Question: {}".format(i["question"]))
    print("\n")
    print("*" * 50)
    raw_rag_answer, formatted_rag_answer, relevant_docs, timing_info, post_process_function_rag, run_time_rag = retry_hugging_face_inference(answer_with_rag, i["question"], llm, vectorstore_medium_model, PROMPT_RAG, RERANKER, NUMBER_RETRIEVED_DOCS, NUMBER_FINAL_DOCS, post_process_function)
    post_process_function_rag_string = inspect.getsource(post_process_function_rag)
    print("Answer: {}".format(formatted_rag_answer))
    print("\n")
    print("*" * 50)
    rag_answers_large_model_golden_questions.append({"ground_truth": i["answer"], "question": i["question"], "raw_answer": raw_rag_answer, "answer": formatted_rag_answer, "total_run_time": run_time_rag, "timing_info": timing_info, "post_process_function": post_process_function_rag_string, "final_documents": relevant_docs})

/usr/local/lib/python3.11/dist-packages/colbert/utils/amp.py:15: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  return torch.cuda.amp.autocast() if self.activated else NullContextManager()


Question: Who is entitled to Irish citizenship?


**************************************************
=> Retrieving documents...
=> Reranking documents...


100%|██████████| 1/1 [00:00<00:00, 35.62it/s]

=> Generating answer...



/usr/local/lib/python3.11/dist-packages/colbert/utils/amp.py:15: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  return torch.cuda.amp.autocast() if self.activated else NullContextManager()


...


/usr/local/lib/python3.11/dist-packages/colbert/utils/amp.py:15: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  return torch.cuda.amp.autocast() if self.activated else NullContextManager()


Answer: The Cycle to Work Scheme allows an employer to buy a new bike and bike safety equipment up to a value of €3,000 for employees. Employees can then repay the purchase cost to their employer from their gross salary. This document provides information on the scheme, including the types of bikes covered, the equipment covered, and the frequency of use.


**************************************************
Question: What are the operating hours and frequency of the Luas?


**************************************************
=> Retrieving documents...
=> Reranking documents...


100%|██████████| 1/1 [00:00<00:00, 41.37it/s]

=> Generating answer...


Answer: Operating hours vary by stop and direction of service. You can view Luas operating hours and Luas frequency on luas.ie.

There are electronic displays telling you the time of the next tram at every Luas stop. In addition, there are electronic displays and announcements informing passengers of the next stop on board every tram.

Special services operate during Christmas, New Year and on other occasions.


**************************************************


In [ ]:
instance_name = get_instance_name()

In [ ]:
# Gen unique file name for RAG run (note this is only used if only one prompt was run above)
golden_large_model_file_path = generate_json_file_name(PROMPT_TYPE_IN_USE, LLM_IN_USE_NAME, "rag", TEMPERATURE, TOP_K, TOP_P, REPETITION_PENALTY, NO_REPEAT_N_GRAMS, MAX_TOKENS, instance_name, "golden")

In [ ]:
# Initialise results for rag experiment run (again note this cell is only used if one prompt type was used)
rag_experiment_results = {
    "hyperparameters": LLM_HYPERPARAMETERS,
    "model": LLM_IN_USE_NAME,
    "results": rag_answers_large_model_golden_questions,
    "colab_instance": instance_name,
    "prompt_template": prompt_template_rag,
    "prompt_type": PROMPT_TYPE_IN_USE,
    "knowledge_index": "FAISS",
    "reranker": RE_RANKER,
    "num_retrieved_docs": NUMBER_RETRIEVED_DOCS,
    "num_docs_final":  NUMBER_FINAL_DOCS,
    "embeddings": { "model_name": MEDIUM_TOKENISER_MODEL_NAME,
                    "model_kwargs": {"device": device,},
                    "encode_kwargs": {"normalize_embeddings": True, "multi_process": False}
                   },
    "chunk_size": CHUNK_SIZE,
    "chunk_overlap": int(CHUNK_SIZE / 10),
    "add_start_index": True,
    "strip_whitespace": True,
    "separators": MARKDOWN_SEPARATORS,
}

In [ ]:
# Write the single RAG prompt type results to the file
write_json_file(golden_large_model_file_path, rag_experiment_results)

Successfully saved content to golden/golden_results_few_shot_johndennehy101_Mistral-7B-Instruct-v0.3-finetune-irish-citizen-info-v1_NVIDIA_A100-SXM4-40GB_rag_temp0_topk0_topP0_repPenalty0_noRepeatNGrams0_max1000_05-07-2025_14-20-08.json


True

## Golden Dataset - Generate answers with LLM

In [ ]:
# Extract golden dataset from stored json file
with open("golden_dataset.json", "r", encoding="utf-8") as f:
      golden_dataset = json.load(f)


# If set to true, run inference for model for all prompts
if RUN_ALL_PROMPTS:
  print("Running all prompts...")
  print("*" * 100)
  print("\n")
  print(LLM_HYPERPARAMETERS)

  # Loop over prompt types
  for prompt_type, prompt_text in prompt_dict_llm.items():
    print("Running {} prompt type".format(prompt_type))
    print("*" * 100)
    print("\n")
    llm_answers_large_model_golden_questions = []

    # Get prompt type contents for current prompt type
    prompt_template_llm = get_prompt_by_type(prompt_type, prompt_dict_llm)

    # construct prompt template
    PROMPT_LLM = PromptTemplate(
      template=prompt_template_llm, input_variables=["question"]
    )

    # Get post process function for current prompt type
    post_process_function = get_post_process_function_by_prompt_type(prompt_type, post_process_function_dict)

    # For each question in the golden dataset, generate inference responses and write experiment results to json files
    for i in golden_dataset:
      raw_llm_answer, formatted_llm_answer, _, timing_info, post_process_function_llm, run_time_llm = retry_hugging_face_inference(answer_without_rag, i["question"], llm, PROMPT_LLM, post_process_function)
      post_process_function_llm_string = inspect.getsource(post_process_function_llm)
      llm_answers_large_model_golden_questions.append({"ground_truth": i["answer"], "question": i["question"], "raw_answer": raw_llm_answer, "answer": formatted_llm_answer, "total_run_time": run_time_llm, "timing_info": timing_info, "post_process_function": post_process_function_llm_string})

    instance_name = get_instance_name()

    golden_large_model_file_path = generate_json_file_name(prompt_type, LLM_IN_USE_NAME, "llm", TEMPERATURE, TOP_K, TOP_P, REPETITION_PENALTY, NO_REPEAT_N_GRAMS, MAX_TOKENS, instance_name, "golden")

    llm_experiment_results = {
    "hyperparameters": LLM_HYPERPARAMETERS,
    "model": LLM_IN_USE_NAME,
    "results": llm_answers_large_model_golden_questions,
    "colab_instance": instance_name,
    "prompt_template": prompt_template_llm,
    "prompt_type": prompt_type
    }

    write_json_file(golden_large_model_file_path, llm_experiment_results)

# Otherwise, only run for current prompt type
else:
  llm_answers_large_model_golden_questions = []
  
  # Loop over golden dataset, generate outputs for each question and store information in above list for each
  for i in golden_dataset:
    print("Question: {}".format(i["question"]))
    print("\n")
    print("*" * 50)
    raw_llm_answer, formatted_llm_answer, _, timing_info, post_process_function_llm, run_time_llm = retry_hugging_face_inference(answer_without_rag, i["question"], llm, PROMPT_LLM, post_process_function)
    post_process_function_llm_string = inspect.getsource(post_process_function_llm)
    print("Answer: {}".format(formatted_llm_answer))
    print("\n")
    print("*" * 50)
    llm_answers_large_model_golden_questions.append({"ground_truth": i["answer"], "question": i["question"], "raw_answer": raw_llm_answer, "answer": formatted_llm_answer, "timing_info": timing_info, "post_process_function": post_process_function_llm_string, "total_run_time": run_time_llm})

Question: Who is entitled to Irish citizenship?


**************************************************
=> Generating answer...
Answer: - You are entitled to Irish citizenship if you were born in Ireland, or if you were born outside Ireland to an Irish parent.
- If you were born outside Ireland, you may be entitled to Irish citizenship if you were born to an Irish parent and your parent was born in Ireland.
- If you were born outside Ireland, you may be entitled to Irish citizenship if you were born to an Irish parent and your parent was born in Ireland.
- If you were born outside Ireland, you may be entitled to Irish citizenship if you were born to an Irish parent and your parent was born in Ireland.
- If you were born outside Ireland, you may be entitled to Irish citizenship if you were born to an Irish parent and your parent was born in Ireland.
- If you were born outside Ireland, you may be entitled to Irish citizenship if you were born to an Irish parent and your parent was born in I

In [ ]:
instance_name = get_instance_name()

In [ ]:
# Generate unique file path name
golden_llm_large_model_file_path = generate_json_file_name(PROMPT_TYPE_IN_USE, LLM_IN_USE_NAME, "llm", TEMPERATURE, TOP_K, TOP_P, REPETITION_PENALTY, NO_REPEAT_N_GRAMS, MAX_TOKENS, instance_name, "golden")

In [ ]:
# Initialise results variable for LLM run (note this should only be used if only one prompt type was ran)
llm_experiment_results = {
    "hyperparameters": LLM_HYPERPARAMETERS,
    "model": LLM_IN_USE_NAME,
    "results": llm_answers_large_model_golden_questions,
    "colab_instance": instance_name,
    "prompt_template": prompt_template_llm,
    "prompt_type": PROMPT_TYPE_IN_USE
}

In [ ]:
# Write the single prompt type experiment to the json file
write_json_file(golden_llm_large_model_file_path, llm_experiment_results)

Successfully saved content to golden/golden_results_few_shot_johndennehy101_Mistral-7B-Instruct-v0.3-finetune-irish-citizen-info-v1_NVIDIA_A100-SXM4-40GB_llm_temp0_topk0_topP0_repPenalty0_noRepeatNGrams0_max1000_05-07-2025_14-29-09.json


True

## Evaluation

In [ ]:
# Extract all result files for rag within the content directory
rag_results_file_paths = get_file_paths_matching_regex("/content", r"(?=.*golden_results)(?=.*rag)")

In [ ]:
# Extract all result files for llm within the content directory
llm_results_file_paths = get_file_paths_matching_regex("/content", r"(?=.*golden_results)(?=.*llm)")

In [ ]:
# Construct a dict with model results for rag
rag_model_results_dict = {}

for file_path in rag_results_file_paths:
  rag_model_results_contents = read_json_file(file_path)
  rag_model_results_dict[file_path] = rag_model_results_contents

Successfully loaded content from /content/golden/golden_results_few_shot_johndennehy101_Mistral-7B-Instruct-v0.3-finetune-irish-citizen-info-v1_NVIDIA_A100-SXM4-40GB_rag_temp0_topk0_topP0_repPenalty0_noRepeatNGrams0_max1000_05-07-2025_14-20-08.json file


In [ ]:
# Construct a dict with model results for llm
llm_model_results_dict = {}

for file_path in llm_results_file_paths:
  llm_model_results_contents = read_json_file(file_path)
  llm_model_results_dict[file_path] = llm_model_results_contents

Successfully loaded content from /content/golden/golden_results_few_shot_johndennehy101_Mistral-7B-Instruct-v0.3-finetune-irish-citizen-info-v1_NVIDIA_A100-SXM4-40GB_llm_temp0_topk0_topP0_repPenalty0_noRepeatNGrams0_max1000_05-07-2025_14-29-09.json file


### Zero shot classification

In [ ]:
def generate_zero_shot_classification(model, tokeniser, ground_truth: str, generated_text: str):
  """
  Utilise another LLM model to generate zero shot classification for generated text output using the provided ground truth as baseline
  """

  # Tokenise inputs
  inputs = tokeniser(ground_truth, generated_text, return_tensors="pt", truncation=True)

  # Disable gradient calculations to speed inference
  with torch.no_grad():
    logits = model(**inputs).logits

  # Apply softmax to convert logits into probabilites
  probs = F.softmax(logits, dim=1)

  # Extract the label mapping from the configuration
  labels = model.config.id2label

  # Construct dict for probs
  label_probabilities = {label: probs[0][i].item() for i, label in labels.items()}

  # Extract label with max probability value
  max_label = max(label_probabilities, key=label_probabilities.get)

  # Ensure both the most probable label and label probabilities for all are returned to enable comparison
  result = (max_label, label_probabilities)

  return result

In [ ]:
# Initialise zero-shot labelling model and tokeniser
zero_shot_classification_model_name = "facebook/bart-large-mnli"
zero_shot_tokeniser = AutoTokenizer.from_pretrained(zero_shot_classification_model_name)
zero_shot_model = AutoModelForSequenceClassification.from_pretrained(zero_shot_classification_model_name)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

### Run evaluation suite - RAG

In [ ]:
# Run evaluation suite for RAG
for _, rag_model_results_values in rag_model_results_dict.items():
  print(rag_model_results_values)
  rag_model_results_values["zero_shot"] = {
      "model": zero_shot_classification_model_name,
  }

  # for individual_item in rag_model_results_values["/content/golden/golden_results_meta-llama_Llama-3.2-3B-Instruct_NVIDIA_A100-SXM4-40GB_rag_temp1_topk0_topP0_repPenalty0_noRepeatNGrams0_max150_07-06-2025_13-13-28.json"]["results"]:
  for individual_item in rag_model_results_values["results"]:
    # Calculate rouge scores
    rag_rouge_1, rag_rouge_2, rag_rouge_l = generate_rouge_scores(individual_item["ground_truth"], individual_item["answer"])

    # Calculate precision, recall f1 scores
    precision_bert, recall_bert, f1_bert = score([individual_item["answer"]], [individual_item["ground_truth"]], model_type="bert-base-uncased", lang="en", rescale_with_baseline=True)

    # Calculate zero-shot labels
    zero_shot_label_prediction, zero_shot_probabilities = generate_zero_shot_classification(zero_shot_model, zero_shot_tokeniser, individual_item["ground_truth"], individual_item["answer"])

    # Assign results to dict
    individual_item["rouge1_score"] = rag_rouge_1
    individual_item["rouge2_score"] = rag_rouge_2
    individual_item["rougeL_score"] = rag_rouge_l
    individual_item["bert_precision"] = precision_bert.item()
    individual_item["bert_recall"] = recall_bert.item()
    individual_item["bert_f1"] = f1_bert.item()
    individual_item["zero_shot_label"] = zero_shot_label_prediction
    individual_item["zero_shot_probabilities"] = zero_shot_probabilities

  # Generate unique file path name
  golden_evaluation_results_rag_large_model_file_path = generate_json_file_name(rag_model_results_values["prompt_type"], LLM_IN_USE_NAME, "rag", TEMPERATURE, TOP_K, TOP_P, REPETITION_PENALTY, NO_REPEAT_N_GRAMS, MAX_TOKENS, instance_name, "results")
  
  # Write contents to file
  write_json_file(golden_evaluation_results_rag_large_model_file_path, rag_model_results_values)

{'hyperparameters': {'max_new_tokens': 1000}, 'model': 'johndennehy101/Mistral-7B-Instruct-v0.3-finetune-irish-citizen-info-v1', 'results': [{'ground_truth': 'You are entitled to claim Irish citizenship if any one of the following applies: 1. You were born in Ireland or Northern Ireland before 1 January 2005 2. You were born in Ireland or Northern Ireland after 31 December 2004 and any of the following applies to you: One (or both) of your parents was Irish or entitled to be an Irish citizen or a British citizen at the time of your birth One (or both) of your parents was an Irish or British citizen and died before you were born One (or both) of your parents was entitled to live in Ireland or Northern Ireland without any restriction on their period of residency One (or both) of your parents was legally resident on the island of Ireland for 3 out of the 4 years immediately before your birth (this does not include residence on a student visa, or residence while awaiting an international p

In [ ]:
golden_evaluation_results_rag_large_model_file_path = generate_json_file_name(PROMPT_TYPE_IN_USE, LLM_IN_USE_NAME, "rag", TEMPERATURE, TOP_K, TOP_P, REPETITION_PENALTY, NO_REPEAT_N_GRAMS, MAX_TOKENS, instance_name, "results")

In [ ]:
write_json_file(golden_evaluation_results_rag_large_model_file_path, rag_model_results_dict)

Successfully saved content to results/golden_results_instruction_mistralai_Mistral-7B-Instruct-v0.3_NVIDIA_A100-SXM4-40GB_rag_temp0_topk0_topP0_repPenalty0_noRepeatNGrams0_max0_17-06-2025_09-17-25.json


True

### Run evaluation suite - LLM

In [ ]:
# Run evaluation suite for LLM
for _, llm_model_results_values in llm_model_results_dict.items():
  print(llm_model_results_values)
  llm_model_results_values["zero_shot"] = {
      "model": zero_shot_classification_model_name,
  }
  # for individual_item in llm_model_results_values["/content/golden/golden_results_meta-llama_Llama-3.2-3B-Instruct_NVIDIA_A100-SXM4-40GB_llm_temp1_topk0_topP0_repPenalty0_noRepeatNGrams0_max150_07-06-2025_13-15-58.json"]["results"]:
  for individual_item in llm_model_results_values["results"]:
    # Generate ROUGE scores
    llm_rouge_1, llm_rouge_2, llm_rouge_l = generate_rouge_scores(individual_item["ground_truth"], individual_item["answer"])

    # Generate precision, recall, F1 scores
    precision_bert, recall_bert, f1_bert = score([individual_item["answer"]], [individual_item["ground_truth"]], model_type="bert-base-uncased", lang="en", rescale_with_baseline=True)

    # Generate zero-shot labels
    zero_shot_label_prediction, zero_shot_probabilities = generate_zero_shot_classification(zero_shot_model, zero_shot_tokeniser, individual_item["ground_truth"], individual_item["answer"])

    # Assign result values to individual items
    individual_item["rouge1_score"] = llm_rouge_1
    individual_item["rouge2_score"] = llm_rouge_2
    individual_item["rougeL_score"] = llm_rouge_l
    individual_item["bert_precision"] = precision_bert.item()
    individual_item["bert_recall"] = recall_bert.item()
    individual_item["bert_f1"] = f1_bert.item()
    individual_item["zero_shot_label"] = zero_shot_label_prediction
    individual_item["zero_shot_probabilities"] = zero_shot_probabilities
  
  # Generate unique file path name
  golden_evaluation_results_llm_large_model_file_path = generate_json_file_name(llm_model_results_values["prompt_type"], LLM_IN_USE_NAME, "llm", TEMPERATURE, TOP_K, TOP_P, REPETITION_PENALTY, NO_REPEAT_N_GRAMS, MAX_TOKENS, instance_name, "results")
  
  # Write contents to results file
  write_json_file(golden_evaluation_results_llm_large_model_file_path, llm_model_results_values)

{'hyperparameters': {'max_new_tokens': 1000}, 'model': 'johndennehy101/Mistral-7B-Instruct-v0.3-finetune-irish-citizen-info-v1', 'results': [{'ground_truth': 'You are entitled to claim Irish citizenship if any one of the following applies: 1. You were born in Ireland or Northern Ireland before 1 January 2005 2. You were born in Ireland or Northern Ireland after 31 December 2004 and any of the following applies to you: One (or both) of your parents was Irish or entitled to be an Irish citizen or a British citizen at the time of your birth One (or both) of your parents was an Irish or British citizen and died before you were born One (or both) of your parents was entitled to live in Ireland or Northern Ireland without any restriction on their period of residency One (or both) of your parents was legally resident on the island of Ireland for 3 out of the 4 years immediately before your birth (this does not include residence on a student visa, or residence while awaiting an international p

In [ ]:
golden_evaluation_results_llm_large_model_file_path = generate_json_file_name(PROMPT_TYPE_IN_USE, LLM_IN_USE_NAME, "llm", TEMPERATURE, TOP_K, TOP_P, REPETITION_PENALTY, NO_REPEAT_N_GRAMS, MAX_TOKENS, instance_name, "results")

In [ ]:
write_json_file(golden_evaluation_results_llm_large_model_file_path, llm_model_results_dict)

Successfully saved content to results/golden_results_instruction_mistralai_Mistral-7B-Instruct-v0.3_NVIDIA_A100-SXM4-40GB_llm_temp0_topk0_topP0_repPenalty0_noRepeatNGrams0_max0_17-06-2025_09-18-02.json


True

In [ ]:
# Tidy up so that new iteration can be run without requiring reload of full notebook
delete_model(llm)

In [ ]:
# Tidy up file directories to remove generated outpus
delete_files_in_directory("/content/golden")
delete_files_in_directory("/content/results")

Deleted 1 files from /content/golden directory
Deleted 1 files from /content/results directory
